#### Libraries importing

In [263]:
import xml.etree.ElementTree as ET
import pandas as pd
import random
import string
from xml.dom import minidom
from datetime import datetime
import numpy as np
import time
import inspect
import ast
import types
import uuid


#### Classes Definitions

In [276]:
class UMLClass:
    """Class to store UML Class details."""
    def __init__(self, class_id, name):
        self.class_id = class_id
        self.name = name
        self.generalizations = []
        self.specializations = []

#### Available functions
##### Parsing XML to extract classes and relations

In [265]:
def parseClasses(root):
    """
    Parse the XML to extract classes and return a dictionary of class IDs and names.
    
    Args:
        root: XML root element.
        
    Returns:
        classes: A dictionary of UMLClass objects.
        df_classes: DataFrame of class IDs and names.
    """
    classes = {}

    for class_element in root.findall(".//Class"):
        class_id = class_element.get("Id")
        class_name = class_element.get("Name")

        if class_id and class_id not in classes and class_name not in [cls.name for cls in classes.values()]:
            new_class = UMLClass(class_id=class_id, name=class_name)
            classes[class_id] = new_class

    class_data = {
        'Class ID': [cls.class_id for cls in classes.values()],
        'Class Name': [cls.name for cls in classes.values()]
    }

    df_classes = pd.DataFrame(class_data)
    return classes, df_classes

In [266]:
def findAssociations(root, classes):
    """
    Extract association, aggregation, and composition relationships from the XML.
    
    Args:
        root: XML root element.
        classes: Dictionary of UMLClass objects.

    Returns:
        relationships: A list of relationship dictionaries.
    """
    relationships = []

    for association_element in root.findall(".//Association"):
        from_end_element = association_element.find(".//FromEnd/AssociationEnd")
        to_end_element = association_element.find(".//ToEnd/AssociationEnd")
        
        if from_end_element is not None and to_end_element is not None:
            from_class_id = from_end_element.get("EndModelElement")
            to_class_id = to_end_element.get("EndModelElement")

            if from_class_id and to_class_id:
                from_class_name = classes.get(from_class_id, UMLClass(from_class_id, "Unknown")).name
                to_class_name = classes.get(to_class_id, UMLClass(to_class_id, "Unknown")).name

                aggregation = from_end_element.get("AggregationKind", "None")
                if aggregation == "Shared":
                    relationship_type = "Aggregation"
                elif aggregation == "Composite":
                    relationship_type = "Composition"
                else:
                    relationship_type = "Association"

                relationships.append({
                    'Relationship Type': relationship_type,
                    'From Class ID': from_class_id,
                    'From Class Name': from_class_name,
                    'To Class ID': to_class_id,
                    'To Class Name': to_class_name
                })

    return relationships

In [267]:
def findGeneralizations(root, classes, relationships):
    """
    Extract generalization relationships from the XML.
    
    Args:
        root: XML root element.
        classes: Dictionary of UMLClass objects.
        relationships: List of existing relationships (to append to).
    
    Returns:
        relationships: Updated list of relationships including generalizations.
    """
    for generalization_element in root.findall(".//Generalization"):
        from_class_id = generalization_element.get("From")
        to_class_id = generalization_element.get("To")

        if from_class_id and to_class_id:
            from_class_name = classes.get(from_class_id, UMLClass(from_class_id, "Unknown")).name
            to_class_name = classes.get(to_class_id, UMLClass(to_class_id, "Unknown")).name

            relationships.append({
                'Relationship Type': 'Generalization',
                'From Class ID': from_class_id,
                'From Class Name': from_class_name,
                'To Class ID': to_class_id,
                'To Class Name': to_class_name
            })
    
    return relationships

In [268]:
def filterUnknownClasses(df):
    """
    Filter out rows from a DataFrame that have 'Unknown' in the class names.
    
    Args:
        df: DataFrame containing relationships.
        
    Returns:
        df: Filtered DataFrame.
    """
    df = df[df['From Class Name'] != 'Unknown']
    df = df[df['To Class Name'] != 'Unknown']
    return df

##### Generation and management of unique ID for classes

In [269]:
def generateId(existingIds, length=10):
    """
    Generates a unique alphanumeric ID of specified length.

    Args:
        existingIds (set): A set of existing IDs to ensure uniqueness.
        length (int): Length of the generated ID. Default is 10.

    Returns:
        str: A unique alphanumeric ID not present in existingIds.
    """
    while True:
        # Generate a random alphanumeric string of the given length
        newId = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        
        # Check if the generated ID is unique within the existing IDs
        if newId not in existingIds:
            return newId  # Return the unique ID

In [270]:
def getIdLength(classes_df):
    """
    Get the length of existing class IDs to ensure new IDs follow the same pattern.

    Args:
        classes_df (pd.DataFrame): DataFrame of CIM classes.

    Returns:
        int: Length of the class IDs.
    """
    return classes_df['Class ID'].apply(len).max()

In [271]:
def getExistingIds(classes_df):
    """
    Get the set of existing class IDs.

    Args:
        classes_df (pd.DataFrame): DataFrame of classes.

    Returns:
        set: Set of existing class IDs to ensure uniqueness.
    """
    # If the DataFrame is empty, return an empty set
    if classes_df.empty:
        return set()
    
    return set(classes_df['Class ID'])

In [272]:
def findClassId(df_classes, class_name):
    """
    Find the class ID of a class by its name.

    Args:
        df_classes (pd.DataFrame): DataFrame of classes.
        class_name (str): The name of the class.

    Returns:
        str: The class ID or None if not found.
    """
    class_id = df_classes[df_classes['Class Name'] == class_name]['Class ID'].values
    return class_id[0] if len(class_id) > 0 else None

##### Finding classes of gen-spec relation

In [273]:
def findGeneralizationChildClasses(relations_df, parent_class_id):
    """
    Find all child classes of a given 'From Class ID' in generalization relationships.

    Args:
        relations_df (pd.DataFrame): DataFrame of relationships.
        parent_class_id (str): The ID of the parent class in the generalization relationship.

    Returns:
        pd.DataFrame: DataFrame of child classes in the generalization relationship.
    """
    return relations_df[(relations_df['Relationship Type'] == 'Generalization') & 
                        (relations_df['From Class ID'] == parent_class_id)]

##### Finding related relationships

In [274]:
def findRelatedRelationships(relations_df, class_id):
    """
    Find relationships (associations, aggregations, compositions) involving a class by its ID.
    
    Args:
        relations_df (pd.DataFrame): DataFrame of relationships.
        class_id (str): ID of the class for which relationships should be found.
        
    Returns:
        pd.DataFrame: DataFrame of found relationships.
    """
    # Find the relationships involving the class
    related_relationships = relations_df[
        (relations_df['From Class ID'] == class_id) | (relations_df['To Class ID'] == class_id)
    ]
    
    # If the result is a list, convert it to a DataFrame
    if isinstance(related_relationships, list):
        related_relationships = pd.DataFrame(related_relationships)

    return related_relationships

##### Mapping related relationships

In [275]:
def mapRelationships(relationships, classMapping, mappedClassId, classesDf):
    """
    Generalize the mapping of relationships from one model to another.

    Args:
        relationships (pd.DataFrame): DataFrame of relationships in the original model.
        classMapping (dict): Dictionary mapping original class names to their transformed counterparts.
        mappedClassId (str): The ID of the corresponding class in the target model.
        classesDf (pd.DataFrame): DataFrame of target model classes (containing the digital counterparts).

    Returns:
        list: A list of relationships mapped to the new model entities.
    """
    new_relationships = []
    
    for _, rel in relationships.iterrows():
        from_id, to_id = None, None
        
        # Handle mapping of 'from' class
        from_class_id = rel['From Class ID']
        from_class_name = rel['From Class Name']
        
        if from_class_id == mappedClassId:
            from_id = mappedClassId
        else:
            # Map 'from' class ID using classMapping
            mapped_from_class_name = classMapping.get(from_class_name, None)
            if mapped_from_class_name:
                from_id = classesDf[classesDf['Class Name'] == mapped_from_class_name]['Class ID'].values
                if len(from_id) == 0:
                    print(f"Warning: Mapped class for {from_class_name} not found.")
                    continue
                from_id = from_id[0]
        
        # Handle mapping of 'to' class
        to_class_id = rel['To Class ID']
        to_class_name = rel['To Class Name']
        
        if to_class_id == mappedClassId:
            to_id = mappedClassId
        else:
            # Map 'to' class ID using classMapping
            mapped_to_class_name = classMapping.get(to_class_name, None)
            if mapped_to_class_name:
                to_id = classesDf[classesDf['Class Name'] == mapped_to_class_name]['Class ID'].values
                if len(to_id) == 0:
                    print(f"Warning: Mapped class for {to_class_name} not found.")
                    continue
                to_id = to_id[0]

        # Only append relationships if both from_id and to_id exist
        if from_id and to_id:
            new_relationships.append({
                'Relationship Type': rel['Relationship Type'],
                'From Class ID': from_id,
                'From Class Name': classesDf[classesDf['Class ID'] == from_id]['Class Name'].values[0],
                'To Class ID': to_id,
                'To Class Name': classesDf[classesDf['Class ID'] == to_id]['Class Name'].values[0],
                'Aggregation': rel.get('Aggregation', None)
            })

    return new_relationships


### Metrics for transformation rules

#### **RULE SIZE METRIC (RSM)**

This metric measures the amount of code within each transformation rule. It reflects the rule's complexity and it is measured in terms of lines of code. 

In [247]:
def calculateSizeOfRule(ruleFunction):
    """
    Calculate the size of a rule by counting the number of logical lines 
    (ignoring comments, docstrings, and blank lines).
    
    Args:
        ruleFunction (function): The rule function to analyze.

    Returns:
        int: The number of logical lines in the function.
    """
    # Get the source code of the function
    sourceCode = inspect.getsource(ruleFunction)
    
    # Parse the source code into an abstract syntax tree (AST)
    tree = ast.parse(sourceCode)

    # Count the number of statements in the AST (ignoring docstrings and comments)
    logicalLines = sum(isinstance(node, (ast.Assign, ast.Call, ast.If, ast.For, ast.While, ast.FunctionDef)) 
                       for node in ast.walk(tree))
    
    return logicalLines

def getFunctionAST(function):
    """Returns the AST of the given function."""
    sourceCode = inspect.getsource(function)
    return ast.parse(sourceCode)

def countLinesOfCode(function):
    """Counts all non-empty, non-comment lines in the function."""
    sourceCode = inspect.getsource(function)
    lines = sourceCode.splitlines()
    
    # Filter out blank lines and comment lines
    codeLines = [line for line in lines if line.strip() and not line.strip().startswith("#")]
    
    return len(codeLines)

def getCalledFunctions(function):
    """
    Get the names of all functions that are called within the given function.
    
    Args:
        function: The function object for which we want to find called functions.
    
    Returns:
        A list of function names (as strings) that are called inside the given function.
    """
    tree = getFunctionAST(function)
    calledFunctions = set()
    
    # Walk through the AST to find function calls
    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            calledFunctions.add(node.func.id)
    
    return list(calledFunctions)

def calculateRecursiveLOC(function, visitedFunctions=None):
    """
    Recursively calculates the lines of code of the given function and its helper functions.
    
    Args:
        function: The starting function for which to calculate the LOC.
        visitedFunctions: A set of function names that have already been counted.
    
    Returns:
        int: The total lines of code including helper functions.
    """
    if visitedFunctions is None:
        visitedFunctions = set()

    # Get the name of the current function
    functionName = function.__name__

    # If this function has already been visited, return 0 to avoid double counting
    if functionName in visitedFunctions:
        return 0

    # Mark the current function as visited
    visitedFunctions.add(functionName)

    # Count the LOC of the current function
    totalLOC = countLinesOfCode(function)

    # Find all functions called by this function
    calledFunctions = getCalledFunctions(function)

    # Recursively count the LOC of each called function
    for funcName in calledFunctions:
        # Try to get the actual function object from the global namespace
        if funcName in globals():
            calledFunction = globals()[funcName]
            if callable(calledFunction):
                totalLOC += calculateRecursiveLOC(calledFunction, visitedFunctions)

    return totalLOC


#### **RULE COMPLEXITY METRIC (RCM)**
Complexity refers to the rule's structural complexity, which can be measured by the number of control structures (loops, conditionals, recursion), branches, and the number of transformations. Count the number of control structures (for, if, etc.) and measure the maximum depth of nested blocks in each rule.

In [248]:
class ComplexityAnalyzer(ast.NodeVisitor):
    """
    AST NodeVisitor to analyze the complexity of a function by counting control structures 
    and measuring the maximum depth of nested blocks.
    """
    def __init__(self):
        self.control_structures = 0
        self.max_depth = 0
        self.current_depth = 0

    def visit(self, node):
        # Check if node is a control structure
        if isinstance(node, (ast.If, ast.For, ast.While, ast.With)):
            self.control_structures += 1
            self.current_depth += 1
            self.max_depth = max(self.max_depth, self.current_depth)
        
        # Visit all child nodes recursively
        self.generic_visit(node)
        
        # Once done with this level, reduce depth
        if isinstance(node, (ast.If, ast.For, ast.While, ast.With)):
            self.current_depth -= 1


def getCalledFunctions(function):
    """
    Get the names of all user-defined functions that are called within the given function.
    
    Args:
        function: The function object for which we want to find called functions.
    
    Returns:
        A list of user-defined function names (as strings) that are called inside the given function.
    """
    tree = ast.parse(inspect.getsource(function))
    calledFunctions = set()

    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            funcName = node.func.id
            # Try to retrieve the function from the global namespace
            if funcName in globals():
                calledFunction = globals()[funcName]
                # Only add user-defined functions (exclude library functions)
                if isinstance(calledFunction, types.FunctionType) and calledFunction.__module__ == '__main__':
                    calledFunctions.add(funcName)
    
    return list(calledFunctions)


def calculateRuleComplexityWithHelpers(ruleFunction, visitedFunctions=None):
    """
    Recursively calculates the complexity of the given function and its helper functions, 
    only considering user-defined functions.
    
    Args:
        ruleFunction (function): The starting function to calculate complexity for.
        visitedFunctions (set): A set of already visited functions to avoid double counting.
    
    Returns:
        tuple: (total number of control structures, maximum depth of nested blocks)
    """
    if visitedFunctions is None:
        visitedFunctions = set()

    # Get the name of the current function
    functionName = ruleFunction.__name__

    # If this function has already been visited, return 0 to avoid double counting
    if functionName in visitedFunctions:
        return 0, 0

    # Mark the current function as visited
    visitedFunctions.add(functionName)

    # Analyze the main function
    analyzer = ComplexityAnalyzer()
    tree = ast.parse(inspect.getsource(ruleFunction))
    analyzer.visit(tree)

    # Initialize the total complexity metrics with the current function's complexity
    totalControlStructures = analyzer.control_structures
    maxDepth = analyzer.max_depth

    # Recursively calculate the complexity of all helper functions
    calledFunctions = getCalledFunctions(ruleFunction)

    for funcName in calledFunctions:
        # Try to get the actual function object from the global namespace
        calledFunction = globals()[funcName]
        if callable(calledFunction):
            subControlStructures, subMaxDepth = calculateRuleComplexityWithHelpers(calledFunction, visitedFunctions)
            totalControlStructures += subControlStructures
            maxDepth = max(maxDepth, subMaxDepth)

    return totalControlStructures, maxDepth


#### **RULE MODULARITY METRIC (RMM)**
Modularity measures how well the transformations are decomposed into smaller, reusable modules. A rule with high modularity should be broken into smaller, independent functions that can be reused across transformations. This metric measures modularity by counting the number of helper functions that are called within the main function. It reflects how decomposed the transformation is.

In [249]:
def calculateModularity(ruleFunction):
    """
    Calculate the modularity of a rule by counting the number of helper functions used.
    
    Args:
        ruleFunction (function): The rule function to analyze.

    Returns:
        int: The number of helper functions called.
    """
    # Get the AST of the function
    tree = ast.parse(inspect.getsource(ruleFunction))

    # Find all function calls (ast.Call) and count unique ones
    functionCalls = set()

    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            functionCalls.add(node.func.id)

    # Count the number of function calls (excluding built-ins like print)
    return len(functionCalls)

#### **RULE DEPENDENCY METRIC (RDM)**
Measures the number of dependencies between transformation rules. A transformation is more complex if it depends on multiple external components, rules, or inputs. Count the number of external functions or data structures that a rule relies on. 
This metric calculates how many external functions or global variables a rule depends on. It measures how many global names are referenced within the function, reflecting the rule’s coupling with external components.

In [250]:
def calculateDependencyMetric(ruleFunction):
    """
    Calculate the number of external dependencies (global functions and variables) 
    used by the rule.
    
    Args:
        ruleFunction (function): The rule function to analyze.

    Returns:
        int: The number of external dependencies.
    """
    # Get the AST of the function
    tree = ast.parse(inspect.getsource(ruleFunction))

    # Collect all global names referenced in the function
    globalNames = set()

    for node in ast.walk(tree):
        if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Load):
            globalNames.add(node.id)

    # Return the number of unique global dependencies (excluding built-ins)
    return len(globalNames)


#### **RULE INPUT/OUTPUT METRIC (RIOM)**
Measures the complexity of the inputs and outputs handled by each transformation rule. A rule that takes many inputs or generates complex outputs may be harder to maintain and understand.
This metric measures the complexity of the rule’s input and output by calculating the number of input parameters and the complexity of the output (size of the transformed classes or relationships).

In [251]:
def calculateIOParameterSizes(ruleFunction, inputParams, *outputParams):
    """
    Calculate the size of I/O parameters (those that are both input and output) before and after the execution of the function.

    Args:
        ruleFunction (function): The rule function to analyze.
        inputParams (list): A list of input data objects (e.g., DataFrames or lists).
        outputParams: The actual outputs returned by the rule function.

    Returns:
        list: A list of dictionaries containing:
              - 'ID': A unique ID for the parameter.
              - 'name': The name of the parameter based on its type.
              - 'size_before': The size of the parameter before function execution.
              - 'size_after': The size of the parameter after function execution.
    """
    def generate_unique_id(existing_ids):
        """
        Generate a unique ID that doesn't exist in the given set of IDs.
        """
        while True:
            new_id = str(uuid.uuid4())
            if new_id not in existing_ids:
                return new_id

    def get_param_name(param):
        """
        Generate a name for the parameter based on its type.
        """
        if isinstance(param, pd.DataFrame):
            return "DataFrame"
        elif isinstance(param, list):
            return "List"
        elif isinstance(param, dict):
            return "Dictionary"
        elif isinstance(param, str):
            return "String"
        else:
            return type(param).__name__

    # Set to track unique IDs
    existing_ids = set()

    # Calculate the size of input parameters before function execution
    inputDetails = {}
    for i, param in enumerate(inputParams):
        paramId = generate_unique_id(existing_ids)
        existing_ids.add(paramId)
        paramName = get_param_name(param)
        paramSizeBefore = len(param) if hasattr(param, '__len__') else 'N/A'
        inputDetails[paramName + str(i)] = {
            'ID': paramId,
            'name': paramName,
            'size_before': paramSizeBefore,
        }

    # Create output parameter details after function execution
    outputDetails = []
    for i, param in enumerate(outputParams):
        paramName = get_param_name(param)
        paramSizeAfter = len(param) if hasattr(param, '__len__') else 'N/A'

        # Check if the output corresponds to an input (by comparing name/type and index)
        if paramName + str(i) in inputDetails:
            outputDetails.append({
                'ID': inputDetails[paramName + str(i)]['ID'],  # Use the same ID as input
                'name': inputDetails[paramName + str(i)]['name'],
                'size_before': inputDetails[paramName + str(i)]['size_before'],
                'size_after': paramSizeAfter
            })

    return outputDetails

### **Reading the CIM (Bologna Domain Model)**

In [277]:
def parseCIMClassesAndRelationships(xmlFile):
    """
    Parse XML file to extract classes and relationships of a CIM model, 
    returning the class and relationship data as DataFrames.
    
    Args:
        xmlFile: Path to the XML file.
    
    Returns:
        df_cim_classes: DataFrame containing CIM classes.
        df_cim_relationships: DataFrame containing CIM relationships.
    """
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    
    # Step 1: Parse classes
    cim_classes, df_cim_classes = parseClasses(root)
    
    # Step 2: Find relationships (associations, aggregations, compositions)
    cim_relationships = findAssociations(root, cim_classes)  
    # Step 3: Find generalizations
    cim_relationships = findGeneralizations(root, cim_classes, cim_relationships)
    # Step 4: Convert relationships to DataFrame
    df_cim_relationships = pd.DataFrame(cim_relationships)
    
    # Step 5: Filter out unknown classes
    df_cim_relationships = filterUnknownClasses(df_cim_relationships)
    return df_cim_classes, df_cim_relationships

In [278]:
cimPath = './cim_vppClassDiagram.xml'
# Parse the XML and output CIM classes and relationships
cimClasses, cimRelations = parseCIMClassesAndRelationships(cimPath)

print("Classes found:\n")
print(cimClasses)

print("\nRelationships found (Associations, Aggregations, Compositions, and Generalizations):")
print(cimRelations.to_string(index=False))


Classes found:

            Class ID      Class Name
0   vlE5zEmGAqAC8Q4g        RealCity
1   uHU5zEmGAqAC8Q43         Bologna
2   XiO5zEmGAqAC8Q5U            Road
3   KBR5zEmGAqAC8Q6O     RoadSegment
4   2fr5zEmGAqAC8Q7p     TrafficLoop
5   UCX5zEmGAqAC8Q74    TrafficLight
6   b0yFzEmGAqAC8Q.m  PhysicalEntity
7   jYzFzEmGAqAC8RDH        Actuator
8   iZbFzEmGAqAC8RDq          Sensor
9   eSQlzEmGAqAC8RE8     TrafficFlow
10  q4tVzEmGAqAC8RKJ  TemporalEntity

Relationships found (Associations, Aggregations, Compositions, and Generalizations):
Relationship Type    From Class ID From Class Name      To Class ID To Class Name
      Aggregation uHU5zEmGAqAC8Q43         Bologna XiO5zEmGAqAC8Q5U          Road
      Aggregation XiO5zEmGAqAC8Q5U            Road KBR5zEmGAqAC8Q6O   RoadSegment
      Aggregation KBR5zEmGAqAC8Q6O     RoadSegment 2fr5zEmGAqAC8Q7p   TrafficLoop
      Aggregation KBR5zEmGAqAC8Q6O     RoadSegment UCX5zEmGAqAC8Q74  TrafficLight
      Association 2fr5zEmGAqAC8Q7p     Traff

### **Transformation rules from CIM to PIM**


#### **RULE1: transformRealSystemToPhysicalTwin** 
*Rule 1* is designed to map the real-world system represented in the Computational Independent Model (CIM) into corresponding "PhysicalTwin" class within the Platform Independent Model (PIM). The "PhysicalTwin" class serves as the physical counterpart with which the Digital Twin interacts, digitally representing physical entities, receiving data through sensors, and sending commands to actuators. This rule transforms the real system class into a unique "PhysicalTwin" class, appending its original name with "PhysicalTwin" and assigning it a unique identifier.


In [254]:
def createPhysicalTwinClasses(childClasses, existingIds, idLength):
    """
    Create new 'PhysicalTwin' corresponding to the real world system.

    Args:
        child_classes (pd.DataFrame): DataFrame of classes representing the real world system.
        existing_ids (set): Set of existing class IDs for uniqueness.
        id_length (int): Length of the generated class IDs.

    Returns:
        pd.DataFrame: DataFrame of newly created 'PhysicalTwin' classes.
    """
    pimClasses = []
    for index, row in childClasses.iterrows():
        childClassName = row['To Class Name']

        # Generate a new unique ID and append 'PhysicalTwin' to the class name
        newId = generateId(existingIds, length=idLength)
        newClassName = childClassName + 'PhysicalTwin'

        # Add the new 'PhysicalTwin' class to the PIM classes list
        pimClasses.append({
            'Class ID': newId,
            'Class Name': newClassName
        })

        # Ensure the new ID is tracked in existingIds to maintain uniqueness
        existingIds.add(newId)

    return pd.DataFrame(pimClasses)

In [255]:
def transformRealSystemToPhysicalTwin(cimClasses, cimRelations, parentClassName):
    """
    Map a system class from CIM to its respective 'PhysicalTwin' classes in PIM.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        parentClassName (str): The name of the parent class from which the child classes are mapped 
                               to their 'PhysicalTwin' counterparts.

    Returns:
        pd.DataFrame: DataFrame of newly created 'PhysicalTwin' classes with unique IDs.
    """
    # Step 1: Get the length of existing class IDs and the list of existing IDs
    idLength = getIdLength(cimClasses)
    existingIds = getExistingIds(cimClasses)

    # Step 2: Find all child classes that have a generalization relationship with the specified parent class
    parentClassId = cimClasses[cimClasses['Class Name'] == parentClassName]['Class ID'].values[0]
    childClasses = findGeneralizationChildClasses(cimRelations, parentClassId)

    # Step 3: Create 'PhysicalTwin' classes for the identified child classes
    dfPimClasses = createPhysicalTwinClasses(childClasses, existingIds, idLength)

    return dfPimClasses

In [279]:
pimClasses = transformRealSystemToPhysicalTwin(cimClasses, cimRelations, 'RealCity')

print("New PhysicalTwin classes created:")
print(pimClasses.to_string(index=False))

sizeRule1 = calculateSizeOfRule(transformRealSystemToPhysicalTwin)
sizeRule1recursive = calculateRecursiveLOC(transformRealSystemToPhysicalTwin)
print(sizeRule1, sizeRule1recursive)


New PhysicalTwin classes created:
        Class ID          Class Name
Sh2iQevB5oiL3Pwm BolognaPhysicalTwin
10 82


#### **RULE2: transformPhysicalEntityToDigitalModel**
This rule aims to map *PhysicalEntity* classes from the Computational Ind. Model (CIM) to their corresponding *digital models* classes in the Platform Independent Model (PIM). These digital models represent the physical entities within the Digital Twin architecture. The relationships between physical entities, such as associations, aggregations, and compositions, are preserved when creating their digital counterparts. After creating the new model classes, a *DigitalModel* class is introduced, which serves as the parent class for all digital models in the DT system, conforming to DT architecture requirements.


In [280]:
def searchPhysicalEntityClass(cimClasses, physicalEntityName):
    """
    Search for the class ID of a real-world system in the CIM classes by its name.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        physicalEntityName (str): The name of the physical entity class to search for.

    Returns:
        str: The class ID of the physical enitty class, or None if not found.
    """
    return findClassId(cimClasses, physicalEntityName)

def searchPhysicalEntities(cimClasses, physicalEntityId, cimRelations):
    """
    Search for all child classes of physical entities type.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        physicalEntityId (str): The ID of the phsyical entity class.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.

    Returns:
        pd.DataFrame: DataFrame of child classes of the phsyical entity class.
    """
    return findGeneralizationChildClasses(cimRelations, physicalEntityId)

def createDigitalModels(cimClasses, cimRelations, existingIds, idLength, physicalEntityName='PhysicalEntity'):
    """
    Create corresponding DigitalModel classes for each real system's child class.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.
        realSystemName (str): The name of the real system to search for.

    Returns:
        list: List of newly created PIM classes (Digital Models).
    """
    # Search for child classes of the real system (e.g., RealCity)
    physicalEntitiesID = searchPhysicalEntityClass(cimClasses, physicalEntityName)
    
    if not physicalEntitiesID:
        print(f"Warning: No RealSystem class found with name '{physicalEntityName}'.")
        return []

    realSystems = searchPhysicalEntities(cimClasses, physicalEntitiesID, cimRelations)
    digitalModels = []

    for _, row in realSystems.iterrows():
        cimClassName = row['To Class Name']
        # Generate new ID and create the DigitalModel class
        newId = generateId(existingIds, idLength)
        newClassName = 'Digital' + cimClassName
        digitalModels.append({
            'Class ID': newId,
            'Class Name': newClassName
        })
        # Add new ID to the existing set to ensure uniqueness
        existingIds.add(newId)

    return digitalModels

def addDigitalModel(existingIds, idLength):
    """
    Add the DigitalModel class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalModel class.
    """
    # Generate a unique ID for the DigitalModel class
    digitalModelId = generateId(existingIds, idLength)
    existingIds.add(digitalModelId)
    return digitalModelId

def addDigitalModelManager(existingIds, idLength):
    """
    Add the DigitalModelManager class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalModelManager class.
    """

    digitalModelManagerId = generateId(existingIds, idLength)
    existingIds.add(digitalModelManagerId)
    return digitalModelManagerId

def createDigitalRelations(cimClasses, cimRelations, newPimClasses, pimRelations):
    """
    Create digital relationships (associations, aggregations, compositions) between digital model classes.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        newPimClasses (pd.DataFrame): DataFrame of newly created digital model classes.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with digital model relationships added.
    """

    newRelationships = []
    processedRelationships = set()  # To track already processed relationships and avoid duplicates

    # Loop through each new digital model class
    for _, newPimClassRow in newPimClasses.iterrows():
        # Extract the original class name by removing 'Model' from the new digital class name
        digitalClassName = newPimClassRow['Class Name']
        originalClassName = digitalClassName.replace('Digital', '')

        # Find the corresponding CIM class with the same original class name
        cimClass = cimClasses[cimClasses['Class Name'] == originalClassName]
        
        # Skip if no matching CIM class is found
        if cimClass.empty:
            print(f"Warning: No corresponding CIM class found for {digitalClassName}.")
            continue

        cimClassId = cimClass['Class ID'].values[0]

        # Find all relationships in CIM where the current CIM class is involved (as either 'from' or 'to')
        relatedCimRelations = findRelatedRelationships(cimRelations, cimClassId)

        # Loop through each related relationship and find the digital counterparts
        for _, relationRow in relatedCimRelations.iterrows():
            fromId, toId = None, None
            relationType = relationRow['Relationship Type']
            aggregationKind = None  # Default to None unless it's aggregation/composition

            # Determine the correct digital model class names for 'from' and 'to'
            if relationRow['From Class ID'] == cimClassId:
                fromId = newPimClassRow['Class ID']
                toClassName = 'Digital' + relationRow['To Class Name']  # Convert CIM class to digital model name
                toId = findClassId(newPimClasses, toClassName)
            elif relationRow['To Class ID'] == cimClassId:
                toId = newPimClassRow['Class ID']
                fromClassName = 'Digital' + relationRow['From Class Name']
                fromId = findClassId(newPimClasses, fromClassName)

            # Skip if either of the digital classes isn't found
            if not fromId or not toId:
                continue

            # Handling different types of relationships based on CIM relationships
            if relationType == 'Aggregation':
                aggregationKind = 'Shared'  # Default is shared aggregation
                relationType = 'Aggregation'  # Both Aggregation and Composition are treated as Aggregation

            elif relationType == 'Composition':
                aggregationKind = 'Composite'  # Composition has aggregation kind 'composite'
                relationType = 'Aggregation'  # Composition is treated as a type of Aggregation

            elif relationType == 'Association':
                aggregationKind = None  # Association doesn't need aggregation kind

            elif relationType == 'Generalization':
                aggregationKind = None  # Generalization doesn't involve aggregation
                relationType = 'Generalization'

            # Create a tuple to avoid duplicates
            relationTuple = (fromId, toId, relationType, aggregationKind)

            if relationTuple not in processedRelationships:
                # Add the new relationship between the digital classes
                newRelationships.append({
                    'Relationship Type': relationType,
                    'From Class ID': fromId,
                    'From Class Name': newPimClasses[newPimClasses['Class ID'] == fromId]['Class Name'].values[0],
                    'To Class ID': toId,
                    'To Class Name': newPimClasses[newPimClasses['Class ID'] == toId]['Class Name'].values[0],
                    'Aggregation': aggregationKind  
                })
                processedRelationships.add(relationTuple)

    # Append the new relationships to the existing PIM relationships
    pimRelations = pd.concat([pimRelations, pd.DataFrame(newRelationships)], ignore_index=True)
    return pimRelations

def addGeneralizationModels(digitalModels, digitalModelID):
    """
    Add generalization relationships between each DigitalShadow and the DigitalShadow class.

    Args:
        digitalShadows (list): List of DigitalShadow classes.
        digitalShadowId (str): The ID of the DigitalShadow class.

    Returns:
        newGeneralizationRelations: 
    """

    newGeneralizationRelations = []
    for model in digitalModels:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': digitalModelID,
            'From Class Name': 'DigitalModel',
            'To Class ID': model['Class ID'],
            'To Class Name': model['Class Name'],
            'Aggregation': None
        })
    return newGeneralizationRelations

def addAggregationModelManager(digitalModelID, digitalModelManagerId, pimRelations):
    """
    Add shared aggregation relationships between each DigitalModel and the DigitalModelManager, checking for duplicates.

    Args:
        digitalModelID (str): The ID of the DigitalModel class.
        digitalModelManagerId (str): The ID of the DigitalModelManager class.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships (with duplicates filtered).
    """
    processedRelations = set()  # Set to track added relationships
    relationTuple = (digitalModelManagerId, digitalModelID, 'Aggregation')
    newAggregationRelation = []

    if relationTuple not in processedRelations:
        newAggregationRelation.append({
            'Relationship Type': 'Aggregation',
            'From Class ID': digitalModelManagerId,
            'From Class Name': 'DigitalModelManager',
            'To Class ID': digitalModelID,
            'To Class Name': 'DigitalModel',
            'Aggregation': 'Shared'
        })
        processedRelations.add(relationTuple)

    # Append the new aggregation relation to pimRelations DataFrame
    newAggregationRelationDf = pd.DataFrame(newAggregationRelation)
    pimRelations = pd.concat([pimRelations, newAggregationRelationDf], ignore_index=True)

    return pimRelations

In [281]:
def transformPhysicalEntityToDigitalRepresentation(cimClasses, cimRelations, pimClasses, pimRelations):
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)
    
    digitalModels = createDigitalModels(cimClasses, cimRelations, existingIds, idLength, 'PhysicalEntity')
    newDigitalModels = pd.DataFrame(digitalModels)
    pimClasses = pd.concat([pimClasses, newDigitalModels], ignore_index=True)
    pimRelations = createDigitalRelations(cimClasses, cimRelations, newDigitalModels, pimRelations)


    digitalModelID = addDigitalModel(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalModelID, 'Class Name': 'DigitalModel'}])], ignore_index=True)
    newGeneralizationRelations = addGeneralizationModels(digitalModels, digitalModelID)
    pimRelations = pd.concat([pimRelations, pd.DataFrame(newGeneralizationRelations)], ignore_index=True)

    digitalModelManagerID = addDigitalModelManager(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalModelManagerID, 'Class Name': 'DigitalModelManager'}])], ignore_index=True)
    pimRelations = addAggregationModelManager(digitalModelID, digitalModelManagerID, pimRelations)

    
    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)

    return pimClasses, pimRelations

In [282]:
pimRelations = pd.DataFrame(columns=["Relationship Type", "From Class ID", "From Class Name", "To Class ID", "To Class Name", "Aggregation"])

inputParams1 = pimClasses
inputParams2 = pimRelations

pimClasses, pimRelations = transformPhysicalEntityToDigitalRepresentation(cimClasses, cimRelations, pimClasses, pimRelations)
print("Updated PIM Classes after transformation rule 2:")
print(pimClasses.to_string(index=False))
print("\nUpdated PIM Relationships after transformation rule 2:")
print(pimRelations.to_string(index=False))

sizeRule2 = calculateSizeOfRule(transformPhysicalEntityToDigitalRepresentation)
sizeRule2recursive = calculateRecursiveLOC(transformPhysicalEntityToDigitalRepresentation)
print(sizeRule2, sizeRule2recursive)
controlStructures, maxDepth = calculateRuleComplexityWithHelpers(transformPhysicalEntityToDigitalRepresentation)
print(f"Number of Control Structures: {controlStructures}, Maximum Depth: {maxDepth}")
modularity = calculateModularity(transformPhysicalEntityToDigitalRepresentation)
print(f"Modularity: {modularity}")
dependency = calculateDependencyMetric(transformPhysicalEntityToDigitalRepresentation)
print(f"Dependency: {dependency}")

riom = calculateIOParameterSizes(transformPhysicalEntityToDigitalRepresentation, [inputParams1, inputParams2], pimClasses, pimRelations)
for param in riom:
    print(f"ID: {param['ID']}, Name: {param['name']}, Size Before: {param['size_before']}, Size After: {param['size_after']}")


Updated PIM Classes after transformation rule 2:
        Class ID          Class Name
Sh2iQevB5oiL3Pwm BolognaPhysicalTwin
8OV7UspaBE6xjnRE         DigitalRoad
TWdtGEuBbwgP7K0m  DigitalRoadSegment
gdeDA0W3USyZ4W4h DigitalTrafficLight
NN2USQVIGNXbJuFz  DigitalTrafficLoop
LH4fCkdjthDRwsMC        DigitalModel
qdIoz72JoQ023tFc DigitalModelManager

Updated PIM Relationships after transformation rule 2:
Relationship Type    From Class ID     From Class Name      To Class ID       To Class Name Aggregation
      Aggregation 8OV7UspaBE6xjnRE         DigitalRoad TWdtGEuBbwgP7K0m  DigitalRoadSegment      Shared
      Aggregation TWdtGEuBbwgP7K0m  DigitalRoadSegment NN2USQVIGNXbJuFz  DigitalTrafficLoop      Shared
      Aggregation TWdtGEuBbwgP7K0m  DigitalRoadSegment gdeDA0W3USyZ4W4h DigitalTrafficLight      Shared
   Generalization LH4fCkdjthDRwsMC        DigitalModel 8OV7UspaBE6xjnRE         DigitalRoad        None
   Generalization LH4fCkdjthDRwsMC        DigitalModel TWdtGEuBbwgP7K0m  Digita

#### **RULE3: transformTemporalEntityToDigitalShadow**

This rule focuses on creating digital shadow entities based on the physical domain's requirements. Digital shadows represent time-series data collected from the physical system and categorized by type. For instance, in the Bologna use case, the *Road* class is a Temporal Entity (through a generalization-specialization relationship), and according to Digital Twin architecture guidelines, a corresponding *RoadShadow* entity will be created. These digital shadows are managed by a *DigitalShadowManager* within the system.

In [283]:
def searchTemporalEntityClass(cimClasses):
    """
    Search for the TemporalEntity class ID in the CIM classes.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.

    Returns:
        str: The class ID of the TemporalEntity class.
    """
    return findClassId(cimClasses, 'TemporalEntity')

def searchTemporalEntities(cimClasses, temporalEntityId, cimRelations):
    """
    Search for all child classes of the TemporalEntity class.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        temporalEntityId (str): The ID of the TemporalEntity class.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.

    Returns:
        pd.DataFrame: DataFrame of child classes of the TemporalEntity.
    """
    return findGeneralizationChildClasses(cimRelations, temporalEntityId)

def createDigitalShadows(cimClasses, cimRelations, existingIds, idLength):
    """
    Create corresponding DigitalShadow classes for each TemporalEntity child class.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        list: List of newly created PIM classes (Digital Shadows).
    """
    # Search for child classes of TemporalEntity
    temporalEntityId = searchTemporalEntityClass(cimClasses)
    temporalEntities = searchTemporalEntities(cimClasses, temporalEntityId, cimRelations)
    digitalShadows = []

    for _, row in temporalEntities.iterrows():
        cimClassName = row['To Class Name']
        # Generate new ID and create the shadow class
        newId = generateId(existingIds, idLength)
        newClassName = cimClassName + 'Shadow'
        digitalShadows.append({
            'Class ID': newId,
            'Class Name': newClassName
        })
        # Add new ID to the existing set to ensure uniqueness
        existingIds.add(newId)

    return digitalShadows

def addDigitalShadow(existingIds, idLength):
    """
    Add the DigitalShadow class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalShadow class.
    """
    digitalShadowId = generateId(existingIds, idLength)
    existingIds.add(digitalShadowId)
    return digitalShadowId

def addDigitalShadowManager(existingIds, idLength):
    """
    Add the DigitalShadowManager class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalShadowManager class.
    """
    digitalShadowManagerId = generateId(existingIds, idLength)
    existingIds.add(digitalShadowManagerId)
    return digitalShadowManagerId

def addAggregationDataTrace(pimClasses, digitalShadows, existingIds, idLength, pimRelations):
    """
    Add DigitalDataTrace class and establish composite aggregation relationships, while checking for duplicates.
    
    Args:
        pimClasses (pd.DataFrame): DataFrame of existing PIM classes.
        digitalShadows (list): List of DigitalShadow classes.
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.
    
    Returns:
        str: ID of DigitalDataTrace class.
        pd.DataFrame: Updated PIM relationships (with duplicates filtered).
    """
    # Create DigitalDataTrace class
    digitalDataTraceId = generateId(existingIds, idLength)
    digitalDataTrace = pd.DataFrame([{
        'Class ID': digitalDataTraceId,
        'Class Name': 'DigitalDataTrace'
    }])
    existingIds.add(digitalDataTraceId)

    # Add DigitalDataTrace to pimClasses
    pimClasses = pd.concat([pimClasses, digitalDataTrace], ignore_index=True)

    # Add composite aggregation relationship between each DigitalShadow and DigitalDataTrace
    newRelations = []
    processedRelations = set()  # Set to track added relationships
    for shadow in digitalShadows:
        relation_tuple = (shadow['Class ID'], digitalDataTraceId, 'Aggregation')
        if relation_tuple not in processedRelations:
            newRelations.append({
                'Relationship Type': 'Aggregation',
                'From Class ID': shadow['Class ID'],
                'From Class Name': shadow['Class Name'],
                'To Class ID': digitalDataTraceId,
                'To Class Name': 'DigitalDataTrace',
                'Aggregation': 'Composite'
            })
            processedRelations.add(relation_tuple)

    # Append the new relationships to pimRelations DataFrame
    newRelationsDf = pd.DataFrame(newRelations)
    pimRelations = pd.concat([pimRelations, newRelationsDf], ignore_index=True)

    return digitalDataTraceId, pimRelations

def addGeneralizationShadows(digitalShadows, digitalShadowId):
    """
    Add generalization relationships between each DigitalShadow and the DigitalShadow class.

    Args:
        digitalShadows (list): List of DigitalShadow classes.
        digitalShadowId (str): The ID of the DigitalShadow class.

    Returns:
        newGeneralizationRelations: 
    """

    newGeneralizationRelations = []
    for shadow in digitalShadows:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': digitalShadowId,
            'From Class Name': 'DigitalShadow',
            'To Class ID': shadow['Class ID'],
            'To Class Name': shadow['Class Name'],
            'Aggregation': None
        })
    return newGeneralizationRelations

def addAggregationManager(digitalShadowID, digitalShadowManagerId, pimRelations):
    """
    Add shared aggregation relationships between each DigitalShadow and the DigitalShadowManager, checking for duplicates.

    Args:
        digitalShadowID (str): The ID of the DigitalShadow class.
        digitalShadowManagerId (str): The ID of the DigitalShadowManager class.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships (with duplicates filtered).
    """
    processedRelations = set()  # Set to track added relationships
    relation_tuple = (digitalShadowManagerId, digitalShadowID, 'Aggregation')
    newAggregationRelation = []

    if relation_tuple not in processedRelations:
        newAggregationRelation.append({
            'Relationship Type': 'Aggregation',
            'From Class ID': digitalShadowManagerId,
            'From Class Name': 'DigitalShadowManager',
            'To Class ID': digitalShadowID,
            'To Class Name': 'DigitalShadow',
            'Aggregation': 'Shared'
        })
        processedRelations.add(relation_tuple)

    # Append the new aggregation relation to pimRelations DataFrame
    newAggregationRelationDf = pd.DataFrame(newAggregationRelation)
    pimRelations = pd.concat([pimRelations, newAggregationRelationDf], ignore_index=True)

    return pimRelations


In [284]:
def transformTemporalEntityToDigitalShadow(cimClasses, cimRelations, pimClasses, pimRelations):
    """
    Transforms Temporal Entities into Digital Shadows and manages relationships.
    
    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        pimClasses (pd.DataFrame): DataFrame of PIM classes.
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame, pd.DataFrame: Updated PIM classes and relationships DataFrames.
    """
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    # Create Digital Shadows and add them to PIM classes
    digitalShadows = createDigitalShadows(cimClasses, cimRelations, existingIds, idLength)
    newDigitalShadowsDf = pd.DataFrame(digitalShadows)
    pimClasses = pd.concat([pimClasses, newDigitalShadowsDf], ignore_index=True)

    # Add DigitalDataTrace class and aggregation relationships
    digitalDataTraceID, pimRelations = addAggregationDataTrace(pimClasses, digitalShadows, existingIds, idLength, pimRelations)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalDataTraceID, 'Class Name': 'DigitalDataTrace'}])], ignore_index=True)
    
    # Add DigitalShadow class
    digitalShadowID = addDigitalShadow(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalShadowID, 'Class Name': 'DigitalShadow'}])], ignore_index=True)

    # Add Generalization relationships between DigitalShadows and DigitalShadow class
    newGeneralizationRelations = addGeneralizationShadows(digitalShadows, digitalShadowID)
    pimRelations = pd.concat([pimRelations, pd.DataFrame(newGeneralizationRelations)], ignore_index=True)

    # Add DigitalShadowManager and its relationships
    digitalShadowManagerID = addDigitalShadowManager(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalShadowManagerID, 'Class Name': 'DigitalShadowManager'}])], ignore_index=True)

    # Add shared aggregation relationships between DigitalShadows and DigitalShadowManager
    pimRelations = addAggregationManager(digitalShadowID, digitalShadowManagerID, pimRelations)

    # Remove duplicates from PIM relationships and classes
    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)

    return pimClasses, pimRelations

In [285]:
# testing
start_time = time.time()
inputParams1 = pimClasses
inputParams2 = pimRelations
pimClasses, pimRelations = transformTemporalEntityToDigitalShadow(cimClasses, cimRelations, pimClasses, pimRelations)
end_time = time.time()
print("Updated PIM Classes after transformation rule 3:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 3:")
print(pimRelations.to_string(index=False))

execution_time = end_time - start_time
print(execution_time)

sizeRule3 = calculateSizeOfRule(transformTemporalEntityToDigitalShadow)
sizeRule3recursive = calculateRecursiveLOC(transformTemporalEntityToDigitalShadow)
print(sizeRule3, sizeRule3recursive)

riomr3 = calculateIOParameterSizes(transformPhysicalEntityToDigitalRepresentation, [inputParams1, inputParams2], pimClasses, pimRelations)
for param in riomr3:
    print(f"ID: {param['ID']}, Name: {param['name']}, Size Before: {param['size_before']}, Size After: {param['size_after']}")

Updated PIM Classes after transformation rule 3:
        Class ID           Class Name
Sh2iQevB5oiL3Pwm  BolognaPhysicalTwin
8OV7UspaBE6xjnRE          DigitalRoad
TWdtGEuBbwgP7K0m   DigitalRoadSegment
gdeDA0W3USyZ4W4h  DigitalTrafficLight
NN2USQVIGNXbJuFz   DigitalTrafficLoop
LH4fCkdjthDRwsMC         DigitalModel
qdIoz72JoQ023tFc  DigitalModelManager
uvl9NXse9JgcYndH           RoadShadow
Ti9uDdS6xUv3ScaW    TrafficLoopShadow
Z2XQx1nv8LmBXdre     DigitalDataTrace
Cjrq6N0dtBjCxwJR        DigitalShadow
HLoECxUOVV3e9nNJ DigitalShadowManager

Updated PIM Relationships after transformation rule 3:
Relationship Type    From Class ID      From Class Name      To Class ID       To Class Name Aggregation
      Aggregation 8OV7UspaBE6xjnRE          DigitalRoad TWdtGEuBbwgP7K0m  DigitalRoadSegment      Shared
      Aggregation TWdtGEuBbwgP7K0m   DigitalRoadSegment NN2USQVIGNXbJuFz  DigitalTrafficLoop      Shared
      Aggregation TWdtGEuBbwgP7K0m   DigitalRoadSegment gdeDA0W3USyZ4W4h DigitalTraffi

#### **RULE4: mergeShadowModelFlow**
This rule integrates digital representations by merging the two main components of a Digital Twin system: digital data traces and digital models. Digital data traces are organized into digital shadows, which are managed by a *DigitalShadowManager*. Meanwhile, digital models replicate the physical entities from real-world systems and are managed by a *DigitalModelManager*. 

At the top level, a *DigitalTwinManager* class is introduced, which serves as the central point of control, managing both the *DigitalShadowManager* and the *DigitalModelManager*. At the foundational level, the digital flows are combined through a *DigitalRepresentation* class, which provides an abstract representation of the physical system (or the physical twin in the Digital Twin architecture).

In [286]:
def addDigitalTwinManager(existingIds, idLength):
    """
    Add the DigitalTwinManager class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalTwinManager class.
    """
    digitalTwinManagerId = generateId(existingIds, idLength)
    existingIds.add(digitalTwinManagerId)
    return digitalTwinManagerId

def addDigitalRepresentation(existingIds, idLength):
    """
    Add the DigitalRepresentation class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalRepresentation class.
    """
    digitalRepresentationId = generateId(existingIds, idLength)
    existingIds.add(digitalRepresentationId)
    return digitalRepresentationId

def addAggregationTwinManager(digitalTwinManagerId, digitalShadowManagerId, digitalModelManagerId, pimRelations):
    """
    Add shared aggregation relationships between the DigitalTwinManager and both the DigitalShadowManager 
    and DigitalModelManager classes.

    Args:
        digitalTwinManagerId (str): The ID of the DigitalTwinManager class.
        digitalShadowManagerId (str): The ID of the DigitalShadowManager class.
        digitalModelManagerId (str): The ID of the DigitalModelManager class.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the aggregation relationships added.
    """
    processedRelations = set(pimRelations[['From Class ID', 'To Class ID', 'Relationship Type']].apply(tuple, axis=1))
    newAggregationRelation = []

    # Aggregation between DigitalTwinManager and DigitalShadowManager
    relation_tuple = (digitalTwinManagerId, digitalShadowManagerId, 'Aggregation')
    if relation_tuple not in processedRelations:
        newAggregationRelation.append({
            'Relationship Type': 'Aggregation',
            'From Class ID': digitalTwinManagerId,
            'From Class Name': 'DigitalTwinManager',
            'To Class ID': digitalShadowManagerId,
            'To Class Name': 'DigitalShadowManager',
            'Aggregation': 'Shared'
        })
        processedRelations.add(relation_tuple)

    # Aggregation between DigitalTwinManager and DigitalModelManager
    relation_tuple = (digitalTwinManagerId, digitalModelManagerId, 'Aggregation')
    if relation_tuple not in processedRelations:
        newAggregationRelation.append({
            'Relationship Type': 'Aggregation',
            'From Class ID': digitalTwinManagerId,
            'From Class Name': 'DigitalTwinManager',
            'To Class ID': digitalModelManagerId,
            'To Class Name': 'DigitalModelManager',
            'Aggregation': 'Shared'
        })
        processedRelations.add(relation_tuple)

    # Append the new aggregation relation to pimRelations DataFrame
    newAggregationRelationDf = pd.DataFrame(newAggregationRelation)
    pimRelations = pd.concat([pimRelations, newAggregationRelationDf], ignore_index=True)

    return pimRelations

def addGeneralizationRepresentation(digitalRepresentationId, digitalDataID, digitalModelID, pimRelations):
    """
    Add generalization relationships between DigitalRepresentation and both DigitalDataTrace 
    and DigitalModel classes.

    Args:
        digitalRepresentationId (str): The ID of the DigitalRepresentation class.
        digitalDataID (str): The ID of the DigitalDataTrace class.
        digitalModelID (str): The ID of the DigitalModel class.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the generalization relationships added.
    """

    processedRelations = set(pimRelations[['From Class ID', 'To Class ID', 'Relationship Type']].apply(tuple, axis=1))
    newGeneralizationRelations = []

    relation_tuple = (digitalRepresentationId, digitalDataID, 'Generalization')
    if relation_tuple not in processedRelations:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': digitalRepresentationId,
            'From Class Name': 'DigitalRepresentation',
            'To Class ID': digitalDataID,
            'To Class Name': 'DigitalDataTrace',
            'Aggregation': None
        })
        processedRelations.add(relation_tuple)

    relation_tuple = (digitalRepresentationId, digitalModelID, 'Generalization')
    if relation_tuple not in processedRelations:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': digitalRepresentationId,
            'From Class Name': 'DigitalRepresentation',
            'To Class ID': digitalModelID,
            'To Class Name': 'DigitalModel',
            'Aggregation': None
        })
        processedRelations.add(relation_tuple)

    newGeneralizationRelationDf = pd.DataFrame(newGeneralizationRelations)
    pimRelations = pd.concat([pimRelations, newGeneralizationRelationDf], ignore_index=True)
    return pimRelations


In [287]:
def mergeShadowModelFlow(cimClasses, cimRelations, pimClasses, pimRelations):
    """
    This function merges the digital representations of the system by combining the Digital Shadow and Digital Model flows.
    
    The digital data traces are grouped into Digital Shadows and managed by a Digital Shadow Manager, 
    while the Digital Models represent the physical entities of the real systems. The flows are merged at the top 
    by introducing a Digital Twin Manager class, which serves as a central access point to both the Digital Shadow Manager 
    and the Digital Model Manager. 

    Additionally, the flows are unified at the bottom by a Digital Representation class, 
    which abstracts the physical system (i.e., the physical twin in the Digital Twin architecture). 
    The Digital Model and Digital Data Trace classes, representing the structural and behavioral aspects of the system, 
    are connected to the Digital Representation class through generalization-specialization relationships, 
    where Digital Representation is the parent class.

    Args:
        cimClasses (pd.DataFrame): The CIM classes DataFrame representing the original system.
        cimRelations (pd.DataFrame): The CIM relationships DataFrame for the original system.
        pimClasses (pd.DataFrame): The PIM classes DataFrame to which the new digital classes will be added.
        pimRelations (pd.DataFrame): The PIM relationships DataFrame to which the new relationships will be added.

    Returns:
        pd.DataFrame: Updated PIM classes with the newly added Digital Twin, Shadow, and Model elements.
        pd.DataFrame: Updated PIM relationships with the newly added generalization and aggregation relationships.
    """


    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    # Add DigitalTwinManager and DigitalRepresentation
    digitalTwinManagerID = addDigitalTwinManager(existingIds, idLength)
    digitalRepresentationID = addDigitalRepresentation(existingIds, idLength)

    # Add DigitalTwinManager and DigitalRepresentation to pimClasses
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalTwinManagerID, 'Class Name': 'DigitalTwinManager'}])], ignore_index=True)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': digitalRepresentationID, 'Class Name': 'DigitalRepresentation'}])], ignore_index=True)

    # Get DigitalShadowManager and DigitalModelManager IDs
    digitalShadowManagerID = pimClasses[pimClasses['Class Name'] == 'DigitalShadowManager']['Class ID'].values[0]
    digitalModelManagerID = pimClasses[pimClasses['Class Name'] == 'DigitalModelManager']['Class ID'].values[0]

    # Add shared aggregation relationships between DigitalTwinManager, DigitalShadowManager, and DigitalModelManager
    pimRelations = addAggregationTwinManager(digitalTwinManagerID, digitalShadowManagerID, digitalModelManagerID, pimRelations)

    # Get DigitalDataTrace and DigitalModel IDs
    digitalDataID = pimClasses[pimClasses['Class Name'] == 'DigitalDataTrace']['Class ID'].values[0]
    digitalModelID = pimClasses[pimClasses['Class Name'] == 'DigitalModel']['Class ID'].values[0]

    # Add generalization relationships between DigitalRepresentation, DigitalDataTrace, and DigitalModel
    pimRelations = addGeneralizationRepresentation(digitalRepresentationID, digitalDataID, digitalModelID, pimRelations)

    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)
    return pimClasses, pimRelations

In [288]:
start_time = time.time()
pimClasses, pimRelations = mergeShadowModelFlow(cimClasses, cimRelations, pimClasses, pimRelations)
end_time = time.time()
print("Updated PIM Classes after transformation rule 4:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 4:")
print(pimRelations.to_string(index=False))


sizeRule4 = calculateSizeOfRule(mergeShadowModelFlow)
sizeRule4recursive = calculateRecursiveLOC(mergeShadowModelFlow)
print(sizeRule4, sizeRule4recursive)
execution_time = end_time - start_time
print(execution_time)

Updated PIM Classes after transformation rule 4:
        Class ID            Class Name
Sh2iQevB5oiL3Pwm   BolognaPhysicalTwin
8OV7UspaBE6xjnRE           DigitalRoad
TWdtGEuBbwgP7K0m    DigitalRoadSegment
gdeDA0W3USyZ4W4h   DigitalTrafficLight
NN2USQVIGNXbJuFz    DigitalTrafficLoop
LH4fCkdjthDRwsMC          DigitalModel
qdIoz72JoQ023tFc   DigitalModelManager
uvl9NXse9JgcYndH            RoadShadow
Ti9uDdS6xUv3ScaW     TrafficLoopShadow
Z2XQx1nv8LmBXdre      DigitalDataTrace
Cjrq6N0dtBjCxwJR         DigitalShadow
HLoECxUOVV3e9nNJ  DigitalShadowManager
Ss4juLPEcHnmYCjB    DigitalTwinManager
525aOFCjMOZdirbi DigitalRepresentation

Updated PIM Relationships after transformation rule 4:
Relationship Type    From Class ID       From Class Name      To Class ID        To Class Name Aggregation
      Aggregation 8OV7UspaBE6xjnRE           DigitalRoad TWdtGEuBbwgP7K0m   DigitalRoadSegment      Shared
      Aggregation TWdtGEuBbwgP7K0m    DigitalRoadSegment NN2USQVIGNXbJuFz   DigitalTrafficLoop  

#### **RULE5: transformSensorToDataProvider**

This rule focuses on mapping each *sensor* in the physical system to its corresponding *data provider* in the Digital Twin system according to DT architecture specifications. In the Platform Independent Model, each sensor is represented by a *P2DAdapteré class (Physical-to-Digital Adapter) responsible for adapting the physical data flow from the sensors to the digital twin environment.
The P2DAdapter ensures that the data generated by the physical sensors can be consumed by the digital system in a DT compliant format. Since all P2DAdapter classes act as adapters, they are generalized under a common *Adapteré class, which will serve as the parent class in the digital model hierarchy.
This mapping process achieves the integration of physical sensors into the digital twin system, ensuring that physical data is accurately transformed and transmitted to the digital model for processing and analysis.

In [289]:
def searchSensorEntityClass(cimClasses, sensorEntityName):
    """
    Search for the class ID of a sensor entity in the CIM classes by its name.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        sensorEntityName (str): The name of the sensor entity class to search for.

    Returns:
        str: The class ID of the sensor entity class, or None if not found.
    """
    return findClassId(cimClasses, sensorEntityName)

def searchSensorEntities(cimClasses, sensorEntityId, cimRelations):
    """
    Search for all child classes of the sensor entity type.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        sensorEntityId (str): The ID of the sensor entity class.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.

    Returns:
        pd.DataFrame: DataFrame of child classes of the sensor entity class.
    """
    return findGeneralizationChildClasses(cimRelations, sensorEntityId)

def addP2DAdapters(dataProviders, existingIds, idLength, pimClasses):
    """
    Add P2DAdapter classes for each sensor/data provider to the PIM model. 
    If there's only one data provider, use a generic name 'P2DAdapter'. 
    If there are multiple providers, the adapter name will be the provider's name 
    with 'DataProvider' removed.

    Args:
        dataProviders (list): List of data providers classes to which P2DAdapters will be added.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.
        pimClasses (pd.DataFrame): DataFrame of PIM classes to which the new adapters will be added.

    Returns:
        pd.DataFrame: Updated PIM classes with the newly created P2DAdapters.
    """
    newClasses = []
    
    # If there's only one provider, use the generic name 'P2DAdapter'
    if len(dataProviders) == 1:
        newAdapterID = generateId(existingIds, idLength)
        existingIds.add(newAdapterID)
        newClasses.append({
            'Class ID': newAdapterID,
            'Class Name': 'P2DAdapter'
        })
    else:
        # If there are multiple providers, create unique adapter names
        for provider in dataProviders:
            newAdapterID = generateId(existingIds, idLength)
            existingIds.add(newAdapterID)
            # Remove 'DataProvider' from the provider name
            className = provider['Class Name'].replace('DataProvider', '')
            newClasses.append({
                'Class ID': newAdapterID,
                'Class Name': 'P2DAdapter' + className
            })

    # Append new adapter classes to PIM classes DataFrame
    pimClasses = pd.concat([pimClasses, pd.DataFrame(newClasses)], ignore_index=True)
    
    return newClasses, pimClasses

def createDataProviders(cimClasses, cimRelations, existingIds, idLength, sensorEntityName='Sensor'):
    """
    Create digital data providers for each sensor entity found in the CIM classes.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.
        sensorEntityName (str, optional): The name of the sensor entity to search for (default is 'Sensor').

    Returns:
        list: List of newly created data provider classes (for sensors).
    """
    # Search for the sensor entity class in CIM classes
    sensorID = searchSensorEntityClass(cimClasses, sensorEntityName)
    if not sensorID:
        print(f"Warning: No Sensor entity class found with name '{sensorEntityName}'.")
        return []
    
    # Search for all child entities of the sensor class
    sensorEntities = searchSensorEntities(cimClasses, sensorID, cimRelations)
    dataProviders = []

    # Create data receivers for each sensor entity
    for _, row in sensorEntities.iterrows():
        cimClassName = row['To Class Name']
        newId = generateId(existingIds, idLength)
        newClassName = cimClassName + 'DataProvider'
        dataProviders.append({
            'Class ID': newId,
            'Class Name': newClassName
        })
        existingIds.add(newId)

    return dataProviders

def addAdapter(existingIds, idLength):
    """
    Add the DigitalShadowManager class to the PIM model.

    Args:
        existingIds (set): Set of existing class IDs.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DigitalShadowManager class.
    """
    adapterID = generateId(existingIds, idLength)
    existingIds.add(adapterID)
    return adapterID

def addGeneralizationAdapters(adaptersList, adapterID, pimRelations):
    """
    Add generalization relationships between each DigitalShadow and the DigitalShadow class.

    Args:
        digitalShadows (list): List of DigitalShadow classes.
        digitalShadowId (str): The ID of the DigitalShadow class.

    Returns:
        newGeneralizationRelations: 
    """

    newGeneralizationRelations = []
    for adapter in adaptersList:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': adapterID,
            'From Class Name': 'Adapter',
            'To Class ID': adapter['Class ID'],
            'To Class Name': adapter['Class Name'],
            'Aggregation': None
        })

    newGeneralizationRelationdF = pd.DataFrame(newGeneralizationRelations)
    pimRelations = pd.concat([pimRelations, newGeneralizationRelationdF], ignore_index=True)
    
    return pimRelations

def addUseProviders(providersList, adaptersList, pimRelations):
    """
    Add 'Usage' relationships between the data providers and the adapters in the PIM model.

    Args:
        providersList (list): List of provider classes (DataProviders).
        adaptersList (list): List of adapter classes (P2DAdapters).
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the new 'Usage' relationships added.
    """
    newUseRelations = []

    # Case for a single provider and a single adapter
    if len(providersList) == 1 and len(adaptersList) == 1:
        newUseRelations.append({
            'Relationship Type': 'Usage',
            'From Class ID': providersList[0]['Class ID'],  # Use the first item from the providers list
            'From Class Name': providersList[0]['Class Name'],
            'To Class ID': adaptersList[0]['Class ID'],  # Use the first item from the adapters list
            'To Class Name': adaptersList[0]['Class Name'],
            'Aggregation': None
        })
    else:
        # Multiple providers case
        for provider in providersList:
            providerName = provider['Class Name'].replace('DataProvider', '')
            # Find the corresponding adapter class based on provider name
            adapter = next((adapter for adapter in adaptersList if adapter['Class Name'] == 'P2DAdapter' + (providerName.replace('DataProvider', ''))), None)
            
            if adapter:
                newUseRelations.append({
                    'Relationship Type': 'Usage',
                    'From Class ID': provider['Class ID'],
                    'From Class Name': provider['Class Name'],
                    'To Class ID': adapter['Class ID'],
                    'To Class Name': adapter['Class Name'],
                    'Aggregation': None
                })

    # Append new usage relations to PIM relations DataFrame
    newUseRelationsDf = pd.DataFrame(newUseRelations)
    pimRelations = pd.concat([pimRelations, newUseRelationsDf], ignore_index=True)

    return pimRelations


In [290]:
def transformSensorToProvider(cimClasses, cimRelations, pimClasses, pimRelations):
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    dataProviders=createDataProviders(cimClasses, cimRelations, existingIds, idLength, 'Sensor')
    pimClasses = pd.concat([pimClasses, pd.DataFrame(dataProviders)], ignore_index=True) 

    adaptersList, pimClasses = addP2DAdapters(dataProviders,existingIds,idLength,pimClasses)

    adapterID = addAdapter(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': adapterID, 'Class Name': 'Adapter'}])], ignore_index=True)
    pimRelations = addGeneralizationAdapters(adaptersList, adapterID, pimRelations)

    pimRelations = addUseProviders(dataProviders,adaptersList, pimRelations)

    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)

    return pimClasses, pimRelations

In [291]:
pimClasses, pimRelations = transformSensorToProvider(cimClasses, cimRelations, pimClasses, pimRelations)
print("Updated PIM Classes after transformation rule 5:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 5:")
print(pimRelations.to_string(index=False))

sizeRule5 = calculateSizeOfRule(transformSensorToProvider)
sizeRule5recursive = calculateRecursiveLOC(transformSensorToProvider)
print(sizeRule5, sizeRule5recursive)


Updated PIM Classes after transformation rule 5:
        Class ID              Class Name
Sh2iQevB5oiL3Pwm     BolognaPhysicalTwin
8OV7UspaBE6xjnRE             DigitalRoad
TWdtGEuBbwgP7K0m      DigitalRoadSegment
gdeDA0W3USyZ4W4h     DigitalTrafficLight
NN2USQVIGNXbJuFz      DigitalTrafficLoop
LH4fCkdjthDRwsMC            DigitalModel
qdIoz72JoQ023tFc     DigitalModelManager
uvl9NXse9JgcYndH              RoadShadow
Ti9uDdS6xUv3ScaW       TrafficLoopShadow
Z2XQx1nv8LmBXdre        DigitalDataTrace
Cjrq6N0dtBjCxwJR           DigitalShadow
HLoECxUOVV3e9nNJ    DigitalShadowManager
Ss4juLPEcHnmYCjB      DigitalTwinManager
525aOFCjMOZdirbi   DigitalRepresentation
L6sbK9gAXeunt1mM TrafficLoopDataProvider
BrHGue0r9yPlMeJB              P2DAdapter
ckbqWnFnU33pqjmj                 Adapter

Updated PIM Relationships after transformation rule 5:
Relationship Type    From Class ID         From Class Name      To Class ID        To Class Name Aggregation
      Aggregation 8OV7UspaBE6xjnRE             D

#### **RULE6: transformActuatorToReceiver**

This rule aims at mapping each actuator existing in the physical system to a data receiver within the digital twin system according to DT architecutre specification. For each actatur, as we are in PIM (so platform indepdent model), we will a D2PAdapter class responsible for adapting the digtal flow of data coming from the digital system in a physical system complaint format. Clealry, all the D2P adapters are Adapter, this means that we will add an Adapter class as parent.

In [292]:
def searchActuatorEntityClass(cimClasses, actuatorEntityName):
    """
    Search for the class ID of an actuator entity in the CIM classes by its name.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        actuatorEntityName (str): The name of the actuator entity class to search for.

    Returns:
        str: The class ID of the actuator entity class, or None if not found.
    """
    return findClassId(cimClasses, actuatorEntityName)

def searchActuatorEntities(cimClasses, actuatorEntityId, cimRelations):
    """
    Search for all child classes of the actuator entity type.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        actuatorEntityId (str): The ID of the actuator entity class.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.

    Returns:
        pd.DataFrame: DataFrame of child classes of the actuator entity class.
    """
    return findGeneralizationChildClasses(cimRelations, actuatorEntityId)

def addD2PAdapters(dataReceivers, existingIds, idLength, pimClasses):
    """
    Add D2PAdapter classes for each actuator/data receiver to the PIM model.
    If there's only one data receiver, use a generic name 'D2PAdapter'.
    If there are multiple receivers, the adapter name will be the receiver's name 
    with 'DataReceiver' removed.

    Args:
        dataReceivers (list): List of data receiver classes to which D2PAdapters will be added.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.
        pimClasses (pd.DataFrame): DataFrame of PIM classes to which the new adapters will be added.

    Returns:
        pd.DataFrame: Updated PIM classes with the newly created D2PAdapters.
    """
    newClasses = []
    
    # If there's only one receiver, use the generic name 'D2PAdapter'
    if len(dataReceivers) == 1:
        newAdapterID = generateId(existingIds, idLength)
        existingIds.add(newAdapterID)
        newClasses.append({
            'Class ID': newAdapterID,
            'Class Name': 'D2PAdapter'
        })
    else:
        # If there are multiple receivers, create unique adapter names
        for receiver in dataReceivers:
            newAdapterID = generateId(existingIds, idLength)
            existingIds.add(newAdapterID)
            # Remove 'DataReceiver' from the receiver name
            className = receiver['Class Name'].replace('DataReceiver', '')
            newClasses.append({
                'Class ID': newAdapterID,
                'Class Name': 'D2PAdapter' + className
            })

    # Append new adapter classes to PIM classes DataFrame
    pimClasses = pd.concat([pimClasses, pd.DataFrame(newClasses)], ignore_index=True)
    
    return newClasses, pimClasses

def createDataReceivers(cimClasses, cimRelations, existingIds, idLength, actuatorEntityName='Actuator'):
    """
    Create digital data receivers for each actuator entity found in the CIM classes.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.
        actuatorEntityName (str, optional): The name of the actuator entity to search for (default is 'Actuator').

    Returns:
        list: List of newly created data receiver classes (for actuators).
    """
    # Search for the actuator entity class in CIM classes
    actuatorID = searchActuatorEntityClass(cimClasses, actuatorEntityName)
    if not actuatorID:
        print(f"Warning: No Actuator entity class found with name '{actuatorEntityName}'.")
        return []
    
    # Search for all child entities of the actuator class
    actuatorEntities = searchActuatorEntities(cimClasses, actuatorID, cimRelations)
    dataReceivers = []

    # Create data receivers for each actuator entity
    for _, row in actuatorEntities.iterrows():
        cimClassName = row['To Class Name']
        newId = generateId(existingIds, idLength)
        newClassName = cimClassName + 'DataReceiver'
        dataReceivers.append({
            'Class ID': newId,
            'Class Name': newClassName
        })
        existingIds.add(newId)

    return dataReceivers

def addUseReceivers(receiversList, adaptersList, pimRelations):
    """
    Add 'Usage' relationships between the data receivers and the adapters in the PIM model.

    Args:
        receiversList (list): List of receiver classes (DataReceivers).
        adaptersList (list): List of adapter classes (D2PAdapters).
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the new 'Usage' relationships added.
    """
    newUseRelations = []

    # Case for a single receiver and a single adapter
    if len(receiversList) == 1 and len(adaptersList) == 1:
        newUseRelations.append({
            'Relationship Type': 'Usage',
            'From Class ID': adaptersList[0]['Class ID'],  # Use the first item from the adapters list
            'From Class Name': adaptersList[0]['Class Name'],
            'To Class ID': receiversList[0]['Class ID'],  # Use the first item from the receivers list
            'To Class Name': receiversList[0]['Class Name'],
            'Aggregation': None
        })
    else:
        # Multiple receivers case
        for receiver in receiversList:
            receiverName = receiver['Class Name'].replace('DataReceiver', '')
            # Find the corresponding adapter class based on receiver name
            adapter = next((adapter for adapter in adaptersList if adapter['Class Name'] == 'D2PAdapter' + (receiverName.replace('DataReceiver', ''))), None)
            
            if adapter:
                newUseRelations.append({
                    'Relationship Type': 'Usage',
                    'From Class ID': adapter['Class ID'],
                    'From Class Name': adapter['Class Name'],
                    'To Class ID': receiver['Class ID'],
                    'To Class Name': receiver['Class Name'],
                    'Aggregation': None
                })

    # Append new usage relations to PIM relations DataFrame
    newUseRelationsDf = pd.DataFrame(newUseRelations)
    pimRelations = pd.concat([pimRelations, newUseRelationsDf], ignore_index=True)

    return pimRelations


In [293]:
def transformActuatorToReceiver(cimClasses, cimRelations, pimClasses, pimRelations):
    """
    Transform actuators from the CIM model to data receivers in the PIM model with D2PAdapters.
    
    This includes adding D2PAdapter classes, the 'Adapter' superclass, and usage and generalization relationships.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        pimClasses (pd.DataFrame): DataFrame of existing PIM classes.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM classes and relationships.
    """
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    # Create data receivers for actuators
    dataReceivers = createDataReceivers(cimClasses, cimRelations, existingIds, idLength, 'Actuator')
    pimClasses = pd.concat([pimClasses, pd.DataFrame(dataReceivers)], ignore_index=True) 

    # Add D2PAdapter classes
    adaptersList, pimClasses = addD2PAdapters(dataReceivers, existingIds, idLength, pimClasses)

    # Check if 'Adapter' superclass already exists
    if not pimClasses[pimClasses['Class Name'] == 'Adapter'].empty:
        adapterID = pimClasses[pimClasses['Class Name'] == 'Adapter']['Class ID'].values[0]
    else:
        # Otherwise create it
        adapterID = addAdapter(existingIds, idLength)
        pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': adapterID, 'Class Name': 'Adapter'}])], ignore_index=True)

    # Add generalization relationships between adapters and 'Adapter'
    pimRelations = addGeneralizationAdapters(adaptersList, adapterID, pimRelations)

    # Add usage relationships between data receivers and adapters
    pimRelations = addUseReceivers(dataReceivers, adaptersList, pimRelations)

    # Remove duplicates from the PIM classes and relations
    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)

    return pimClasses, pimRelations


In [294]:
pimClasses, pimRelations = transformActuatorToReceiver(cimClasses, cimRelations, pimClasses, pimRelations)
print("Updated PIM Classes after transformation rule 6:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 6:")
print(pimRelations.to_string(index=False))

sizeRule6 = calculateSizeOfRule(transformActuatorToReceiver)
sizeRule6recursive = calculateRecursiveLOC(transformActuatorToReceiver)
print(sizeRule6, sizeRule6recursive)

Updated PIM Classes after transformation rule 6:
        Class ID               Class Name
Sh2iQevB5oiL3Pwm      BolognaPhysicalTwin
8OV7UspaBE6xjnRE              DigitalRoad
TWdtGEuBbwgP7K0m       DigitalRoadSegment
gdeDA0W3USyZ4W4h      DigitalTrafficLight
NN2USQVIGNXbJuFz       DigitalTrafficLoop
LH4fCkdjthDRwsMC             DigitalModel
qdIoz72JoQ023tFc      DigitalModelManager
uvl9NXse9JgcYndH               RoadShadow
Ti9uDdS6xUv3ScaW        TrafficLoopShadow
Z2XQx1nv8LmBXdre         DigitalDataTrace
Cjrq6N0dtBjCxwJR            DigitalShadow
HLoECxUOVV3e9nNJ     DigitalShadowManager
Ss4juLPEcHnmYCjB       DigitalTwinManager
525aOFCjMOZdirbi    DigitalRepresentation
L6sbK9gAXeunt1mM  TrafficLoopDataProvider
BrHGue0r9yPlMeJB               P2DAdapter
ckbqWnFnU33pqjmj                  Adapter
ddDwSBOXW3Q4LdMp TrafficLightDataReceiver
V9ajJ3yTb1RSkKSS               D2PAdapter

Updated PIM Relationships after transformation rule 6:
Relationship Type    From Class ID         From Class N

#### **RULE7: integrateServiceAndFeedback**

This rule introduces the *ServiceManager* class, which is responsible for managing the services offered by the Digital Twin system. The ServiceManager has a usage dependency on the DigitalTwinManager, allowing it to leverage digital twin models fed with real-world data to offer system-wide services. Additionally, the ServiceManager contains *Feedback Providers*, one for each data receiver present in the physical system. For each data receiver, there is a corresponding Feedback Provider. These feedback providers interact with D2PAdapters (digital-to-physical adapters) to send commands to the physical world via the connected adapter, thus facilitating communication from the digital twin system to the physical world.

In [296]:
def addServiceManager(existingIds, idLength):
    """
    Add a ServiceManager class to the PIM model.
    
    The ServiceManager class is responsible for managing service-related feedback and interactions
    between various components within the Digital Twin architecture.

    Args:
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created ServiceManager class.
    """
    serviceID = generateId(existingIds, idLength)
    existingIds.add(serviceID)
    return serviceID

def addFeedback(existingIds, idLength):
    """
    Add a Feedback class to the PIM model.

    Feedback represents a mechanism that collects feedback from data receivers to monitor and adjust
    the behavior of the physical system in the Digital Twin architecture.

    Args:
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created Feedback class.
    """
    feedbackID = generateId(existingIds, idLength)
    existingIds.add(feedbackID)
    return feedbackID

def createFeedbackProviders(pimClasses, existingIds, idLength):
    """
    Create Feedback classes for each DataReceiver class in the PIM model.
    
    Feedback providers are added for all classes identified as data receivers in the system.
    These feedback providers will serve as feedback channels for system data.

    Args:
        pimClasses (pd.DataFrame): DataFrame of PIM classes.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        tuple: A list of new Feedback providers and the updated PIM classes DataFrame.
    """
    dataReceivers = pimClasses[pimClasses['Class Name'].str.contains('DataReceiver')]

    newFeedbackProviders = []
    for _, receiver in dataReceivers.iterrows():
        feedbackID = addFeedback(existingIds, idLength)
        newFeedbackProviders.append({
            'Class ID': feedbackID,
            'Class Name': 'Feedback' + receiver['Class Name'].replace('DataReceiver', '')
        })

    pimClasses = pd.concat([pimClasses, pd.DataFrame(newFeedbackProviders)], ignore_index=True)

    return newFeedbackProviders, pimClasses

def addAggregationFeedback(feedbackList, serviceID, pimRelations):
    """
    Add aggregation relationships between Feedback classes and the ServiceManager class.

    This function establishes an aggregation relationship, specifically a composite aggregation, 
    between the ServiceManager class and each Feedback provider class.

    Args:
        feedbackList (list): List of Feedback classes.
        serviceID (str): The class ID of the ServiceManager.
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the new aggregation relations added.
    """
    newRelations = []
    processedRelations = set()  # Set to track added relationships

    for feedback in feedbackList:
        relation_tuple = (feedback['Class ID'], serviceID, 'Aggregation')
        if relation_tuple not in processedRelations:
            newRelations.append({
                'Relationship Type': 'Aggregation',
                'From Class ID': serviceID,
                'From Class Name': 'ServiceManager',
                'To Class ID': feedback['Class ID'],
                'To Class Name': feedback['Class Name'],
                'Aggregation': 'Composite'
            })
            processedRelations.add(relation_tuple)

    # Append the new relationships to pimRelations DataFrame
    newRelationsDf = pd.DataFrame(newRelations)
    pimRelations = pd.concat([pimRelations, newRelationsDf], ignore_index=True)

    return pimRelations
    
def addUseService(serviceID, digitalTwinManagerID, pimRelations):
    """
    Add a 'Usage' relationship between the ServiceManager and DigitalTwinManager classes.

    This function creates a usage relationship, where the ServiceManager class uses the 
    DigitalTwinManager class within the Digital Twin architecture.

    Args:
        serviceID (str): The class ID of the ServiceManager.
        digitalTwinManagerID (str): The class ID of the DigitalTwinManager.
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the new 'Usage' relationship added.
    """
    newUseRelations = [{
        'Relationship Type': 'Usage',
        'From Class ID': serviceID,
        'From Class Name': 'ServiceManager',
        'To Class ID': digitalTwinManagerID,
        'To Class Name': 'DigitalTwinManager',
        'Aggregation': None
    }]

    # Append new usage relations to PIM relations DataFrame
    newUseRelationsDf = pd.DataFrame(newUseRelations)
    pimRelations = pd.concat([pimRelations, newUseRelationsDf], ignore_index=True)

    return pimRelations
    
def addUseFeedbackProviders(feedbackList, pimClasses, pimRelations):
    """
    Add 'Usage' relationships between Feedback classes and their corresponding DataReceiver classes.

    This function links each Feedback provider to its corresponding DataReceiver by adding usage 
    relationships in the PIM model.

    Args:
        feedbackList (list): List of Feedback classes.
        pimClasses (pd.DataFrame): DataFrame of PIM classes.
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with the new 'Usage' relationships added.
    """
    newRelations = []
    processedRelations = set()  # Set to track added relationships

    for feedback in feedbackList:
        # Search the PIM class that corresponds to the feedback name without 'Feedback' and with 'DataReceiver'
        feedbackName = feedback['Class Name'].replace('Feedback', '')
        dataReceiver = pimClasses[pimClasses['Class Name'] == (feedbackName + 'DataReceiver')]

        if not dataReceiver.empty:
            dataReceiverID = dataReceiver['Class ID'].values[0]
            dataReceiverName = dataReceiver['Class Name'].values[0]

            relationTuple = (feedback['Class ID'], dataReceiverID, 'Usage')
            if relationTuple not in processedRelations:
                newRelations.append({
                    'Relationship Type': 'Usage',
                    'From Class ID': feedback['Class ID'],
                    'From Class Name': feedback['Class Name'],
                    'To Class ID': dataReceiverID,
                    'To Class Name': dataReceiverName,
                    'Aggregation': None
                })
                processedRelations.add(relationTuple)

    # Append new usage relations to PIM relations DataFrame
    newRelationsDf = pd.DataFrame(newRelations)
    pimRelations = pd.concat([pimRelations, newRelationsDf], ignore_index=True)

    return pimRelations


In [297]:
def integrateServiceAndFeedback(cimClasses, cimRelations, pimClasses, pimRelations): 
    """
    Integrates the ServiceManager and feedback flow into the PIM model, establishing relationships 
    with the DigitalTwinManager and feedback providers.

    This function performs the following operations:
    1. Creates a ServiceManager responsible for managing services offered by the Digital Twin system.
    2. Adds a usage relationship between the ServiceManager and the DigitalTwinManager, 
       allowing the ServiceManager to leverage digital twin models for offering services.
    3. For each data receiver in the physical system, creates a corresponding Feedback Provider class.
       These feedback providers are responsible for sending commands back to the physical world using 
       D2PAdapters connected to data receivers.
    4. Establishes a composite aggregation relationship between the ServiceManager and the 
       Feedback Providers.
    5. Links each Feedback Provider to its respective data receiver via a usage relationship.

    Args:
        cimClasses (pd.DataFrame): DataFrame of CIM classes.
        cimRelations (pd.DataFrame): DataFrame of CIM relationships.
        pimClasses (pd.DataFrame): DataFrame of PIM classes.
        pimRelations (pd.DataFrame): DataFrame of PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM classes and relationships with ServiceManager and feedback flow integrated.
    """

    # Step 1: Get existing class IDs and ID length for unique identification
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    # Step 2: Add the ServiceManager class and append it to the PIM classes
    serviceID = addServiceManager(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': serviceID, 'Class Name': 'ServiceManager'}])], ignore_index=True)

    # Step 3: Create Feedback Providers for each data receiver and append them to the PIM classes
    feedbackList, pimClasses = createFeedbackProviders(pimClasses, existingIds, idLength)

    # Step 4: Establish aggregation relationships between the ServiceManager and Feedback Providers
    pimRelations = addAggregationFeedback(feedbackList, serviceID, pimRelations)

    # Step 5: Retrieve the DigitalTwinManager ID
    digitalTwinManagerID = pimClasses[pimClasses['Class Name'] == 'DigitalTwinManager']['Class ID'].values[0]

    # Step 6: Establish a usage relationship between the ServiceManager and the DigitalTwinManager
    pimRelations = addUseService(serviceID, digitalTwinManagerID, pimRelations)

    # Step 7: Establish usage relationships between Feedback Providers and corresponding Data Receivers
    pimRelations = addUseFeedbackProviders(feedbackList, pimClasses, pimRelations)

    # Step 8: Clean up by removing duplicates in PIM classes and relationships
    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)

    return pimClasses, pimRelations

In [298]:
pimClasses, pimRelations = integrateServiceAndFeedback(cimClasses, cimRelations, pimClasses, pimRelations)
print("Updated PIM Classes after transformation rule 7:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 7:")
print(pimRelations.to_string(index=False))

sizeRule7 = calculateSizeOfRule(integrateServiceAndFeedback)
sizeRule7recursive = calculateRecursiveLOC(integrateServiceAndFeedback)
print(sizeRule4, sizeRule4recursive)

Updated PIM Classes after transformation rule 7:
        Class ID               Class Name
Sh2iQevB5oiL3Pwm      BolognaPhysicalTwin
8OV7UspaBE6xjnRE              DigitalRoad
TWdtGEuBbwgP7K0m       DigitalRoadSegment
gdeDA0W3USyZ4W4h      DigitalTrafficLight
NN2USQVIGNXbJuFz       DigitalTrafficLoop
LH4fCkdjthDRwsMC             DigitalModel
qdIoz72JoQ023tFc      DigitalModelManager
uvl9NXse9JgcYndH               RoadShadow
Ti9uDdS6xUv3ScaW        TrafficLoopShadow
Z2XQx1nv8LmBXdre         DigitalDataTrace
Cjrq6N0dtBjCxwJR            DigitalShadow
HLoECxUOVV3e9nNJ     DigitalShadowManager
Ss4juLPEcHnmYCjB       DigitalTwinManager
525aOFCjMOZdirbi    DigitalRepresentation
L6sbK9gAXeunt1mM  TrafficLoopDataProvider
BrHGue0r9yPlMeJB               P2DAdapter
ckbqWnFnU33pqjmj                  Adapter
ddDwSBOXW3Q4LdMp TrafficLightDataReceiver
V9ajJ3yTb1RSkKSS               D2PAdapter
bXVTTFlRnAJ0R8bx           ServiceManager
1RqCkhZ2XxYoPbof     FeedbackTrafficLight

Updated PIM Relationships 

#### **RULE8: integrateDataManagerAndDataModel**

This rule has to add the DataManager class, responsible for management of data within the Digital Twin system. Moreover, the DataManager has to manage the compliance with a Data Model, i.e the representation of data with respect to a standardized data strcture. In the PIM there should be a data model entity for each digital entity replicating a physical system. Classes like the digitalTwinManager, the Adapter and the serviceManager have a usage relations with the datamanager. P2Dadapter has to be compliant with the data model, meaning data sent from the physical system have to be adapted to respect a data model.

In [299]:
def addDataManager(existingIds, idLength):
    """
    Add a DataManager class to the PIM model.

    The DataManager class is responsible for managing data flow, ensuring proper communication
    between data models and other components of the Digital Twin system.

    Args:
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DataManager class.
    """
    dataManagerID = generateId(existingIds, idLength)
    existingIds.add(dataManagerID)
    return dataManagerID

def addDataModel(existingIds, idLength):
    """
    Add a DataModel class to the PIM model.

    The DataModel class represents the digital models in the Digital Twin architecture that capture
    data about the physical system's behavior and structure.

    Args:
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        str: The class ID of the newly created DataModel class.
    """
    dataModelID = generateId(existingIds, idLength)
    existingIds.add(dataModelID)
    return dataModelID

def createDataModels(pimClasses, existingIds, idLength):
    """
    Create specific DataModel instances based on existing DigitalModel entities in the PIM model.

    This function finds all child classes of the 'DigitalModel' class, and for each, it creates 
    a corresponding DataModel class. The new DataModel class represents data-specific information 
    and relationships, and is given a unique class name derived from the DigitalModel name.

    Args:
        pimClasses (pd.DataFrame): DataFrame of existing PIM classes.
        existingIds (set): Set of existing class IDs to ensure uniqueness.
        idLength (int): Length of generated class IDs.

    Returns:
        tuple: A list of newly created DataModel instances and an updated PIM classes DataFrame.
    """
    # Get the ID of the DigitalModel class
    digitalModelID = pimClasses[pimClasses['Class Name'] == 'DigitalModel']['Class ID'].values[0]
    
    # Find all child classes of DigitalModel (generalization relationships)
    digitalModelsEntities = findGeneralizationChildClasses(pimRelations, digitalModelID)
    
    dataModels = []
    
    # Create a DataModel class for each DigitalModel child entity
    for _, row in digitalModelsEntities.iterrows(): 
        digitalModelName = row['To Class Name']
        digitalModelName = digitalModelName.replace('Digital', '')  # Simplify name
        
        newId = generateId(existingIds, idLength)
        newClassName = digitalModelName + 'DataModel'
        
        dataModels.append({
            'Class ID': newId,
            'Class Name': newClassName
        })
        
        # Add the new ID to ensure uniqueness
        existingIds.add(newId)
    
    # Append new DataModel classes to the PIM classes DataFrame
    newDataModelDf = pd.DataFrame(dataModels)
    pimClasses = pd.concat([pimClasses, newDataModelDf], ignore_index=True)
    
    return dataModels, pimClasses

def addGeneralizationDataModels(dataModels, dataModelID, pimRelations):
    """
    Add generalization relationships between the DataModel class and its child DataModel instances.

    This function creates generalization-specialization relationships between the main DataModel class 
    (acting as the parent) and individual data model instances (acting as the children). Each 
    specialization (child) is linked to the general DataModel class, following the inheritance 
    hierarchy in the PIM model.

    Args:
        dataModels (list): List of data model classes to be linked with the general DataModel class.
        dataModelID (str): The ID of the main DataModel class (acting as the parent in the relationships).
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships with new generalization relationships added.
    """
    newGeneralizationRelations = []
    for datamodel in dataModels:
        newGeneralizationRelations.append({
            'Relationship Type': 'Generalization',
            'From Class ID': dataModelID,
            'From Class Name': 'DataModel',
            'To Class ID': datamodel['Class ID'],
            'To Class Name': datamodel['Class Name'],
            'Aggregation': None
        })

    # Append the new generalization relationships to the pimRelations DataFrame
    newGeneralizationRelationsDf = pd.DataFrame(newGeneralizationRelations)
    pimRelations = pd.concat([pimRelations, newGeneralizationRelationsDf], ignore_index=True)
    
    return pimRelations

def addCompliantWith(dataModelID, dataManagerID, adaptersList, pimRelations):
    """
    Adds 'CompliantWith' relationships between the DataManager, DataModel, and adapters.
    
    Args:
        dataModelID (str): The ID of the DataModel class.
        dataManagerID (str): The ID of the DataManager class.
        adaptersList (list): List of adapter classes.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.
    
    Returns:
        pd.DataFrame: Updated PIM relationships with new 'CompliantWith' relationships added.
    """
    newRelations = []

    # Add CompliantWith relationship between DataManager and DataModel
    newRelations.append({
        'Relationship Type': 'CompliantWith',
        'From Class ID': dataManagerID,
        'From Class Name': 'DataManager',
        'To Class ID': dataModelID,
        'To Class Name': 'DataModel',
        'Aggregation': None
    })

    # Add CompliantWith relationships for each adapter
    for adapter in adaptersList:
        newRelations.append({
            'Relationship Type': 'CompliantWith',
            'From Class ID': adapter['Class ID'],
            'From Class Name': adapter['Class Name'],
            'To Class ID': dataModelID,
            'To Class Name': 'DataModel',
            'Aggregation': None
        })

    # Append new relationships to pimRelations
    newRelationsDf = pd.DataFrame(newRelations)
    pimRelations = pd.concat([pimRelations, newRelationsDf], ignore_index=True)
    
    return pimRelations

def addUseDataManager(dataManagerID, digitalTwinManagerID, serviceManagerID, adapterID, pimRelations):
    """
    Add 'Usage' relationships between the DataManager and other components in the PIM model.
    
    This function creates 'Usage' relationships that connect the DataManager to:
    - The ServiceManager (which uses the DataManager to manage services)
    - The DigitalTwinManager (which uses the DataManager to interface with digital models)
    - The Adapter (which uses the DataManager to communicate with physical systems)

    Args:
        dataManagerID (str): The ID of the DataManager class.
        digitalTwinManagerID (str): The ID of the DigitalTwinManager class.
        serviceManagerID (str): The ID of the ServiceManager class.
        adapterID (str): The ID of the Adapter class.
        pimRelations (pd.DataFrame): DataFrame of existing PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM relationships DataFrame with the newly added 'Usage' relationships.
    """
    
    newUseRelations = [
        {
            'Relationship Type': 'Usage',
            'From Class ID': serviceManagerID,
            'From Class Name': 'ServiceManager',
            'To Class ID': dataManagerID,
            'To Class Name': 'DataManager',
            'Aggregation': None
        },
        {
            'Relationship Type': 'Usage',
            'From Class ID': digitalTwinManagerID,
            'From Class Name': 'DigitalTwinManager',
            'To Class ID': dataManagerID,
            'To Class Name': 'DataManager',
            'Aggregation': None
        },
        {
            'Relationship Type': 'Usage',
            'From Class ID': adapterID,
            'From Class Name': 'Adapter',
            'To Class ID': dataManagerID,
            'To Class Name': 'DataManager',
            'Aggregation': None
        }
    ]
    
    # Append new usage relations to PIM relations DataFrame
    newUseRelationsDf = pd.DataFrame(newUseRelations)
    pimRelations = pd.concat([pimRelations, newUseRelationsDf], ignore_index=True)

    return pimRelations


In [300]:
def integrateDataManagerAndDataModel(pimClasses, pimRelations):
    """
    Integrate the DataManager and DataModel into the PIM model.

    This function introduces both the DataManager and DataModel classes into the Platform Independent Model (PIM). 
    It establishes relationships between these two classes and relevant adapters, ensuring compliance and usage relationships.
    The following steps are executed:
    
    1. **Create and Add DataManager and DataModel Classes**:
        - Adds the `DataManager` class responsible for managing data services in the digital twin system.
        - Adds the `DataModel` class representing the data models associated with the physical system.

    2. **Create Data Models and Generalization Relationships**:
        - Generates specific data model entities for each digital model and adds generalization-specialization relationships between them and the `DataModel`.

    3. **Find Relevant Adapters**:
        - Identifies all `P2DAdapter` classes (representing adapters between physical and digital worlds) in the PIM classes.

    4. **Establish 'CompliantWith' Relationships**:
        - Adds `CompliantWith` relationships between the `DataManager`, `DataModel`, and identified `P2DAdapter` classes, ensuring that these components are compliant with the data manager.

    5. **Add Usage Relationships**:
        - Creates 'Usage' relationships between the `DataManager` and key classes, including `DigitalTwinManager`, `ServiceManager`, and `Adapter`.

    6. **Remove Duplicates**:
        - Ensures there are no duplicate entries in the PIM classes or relationships.

    Args:
        pimClasses (pd.DataFrame): DataFrame containing the current PIM classes.
        pimRelations (pd.DataFrame): DataFrame containing the current PIM relationships.

    Returns:
        pd.DataFrame: Updated PIM classes DataFrame with added DataManager, DataModel, and adapters.
        pd.DataFrame: Updated PIM relationships DataFrame with added generalization, 'CompliantWith', and 'Usage' relationships.
    """
    
    # Step 1: Get existing IDs and length of IDs
    existingIds = getExistingIds(pimClasses)
    idLength = getIdLength(pimClasses)

    # Step 2: Add DataManager and DataModel classes
    dataManagerID = addDataManager(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': dataManagerID, 'Class Name': 'DataManager'}])], ignore_index=True)

    dataModelID = addDataModel(existingIds, idLength)
    pimClasses = pd.concat([pimClasses, pd.DataFrame([{'Class ID': dataModelID, 'Class Name': 'DataModel'}])], ignore_index=True)

    # Step 3: Create data models and generalization relationships
    dataModels, pimClasses = createDataModels(pimClasses, existingIds, idLength)
    pimRelations = addGeneralizationDataModels(dataModels, dataModelID, pimRelations)

    # Step 4: Find all classes that contain 'P2DAdapter' in their class name in pimClasses
    adapterList = pimClasses[pimClasses['Class Name'].str.contains('P2DAdapter')].to_dict('records')

    # Step 5: Add 'CompliantWith' relationships between DataManager, DataModel, and adapters
    pimRelations = addCompliantWith(dataModelID, dataManagerID, adapterList, pimRelations)

    digitalTwinManagerID = pimClasses[pimClasses['Class Name'] == 'DigitalTwinManager']['Class ID'].values[0]
    serviceManagerID = pimClasses[pimClasses['Class Name'] == 'ServiceManager']['Class ID'].values[0]
    adapterID = pimClasses[pimClasses['Class Name'] == 'Adapter']['Class ID'].values[0]
    pimRelations = addUseDataManager(dataManagerID, digitalTwinManagerID, serviceManagerID, adapterID, pimRelations)

    # Step 6: Remove duplicates from the PIM classes and relations
    pimClasses = pimClasses.drop_duplicates(subset='Class Name', keep='first', ignore_index=True)
    pimRelations = pimRelations.drop_duplicates(subset=['From Class Name', 'To Class Name', 'Relationship Type'], keep='first', ignore_index=True)
    
    return pimClasses, pimRelations
    
   
    

In [301]:
pimClasses, pimRelations = integrateDataManagerAndDataModel(pimClasses, pimRelations)
print("Updated PIM Classes after transformation rule 8:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after transformation rule 8:")
print(pimRelations.to_string(index=False))

sizeRule8 = calculateSizeOfRule(integrateDataManagerAndDataModel)
sizeRule8recursive = calculateRecursiveLOC(integrateDataManagerAndDataModel)
print(sizeRule8, sizeRule8recursive)

Updated PIM Classes after transformation rule 8:
        Class ID               Class Name
Sh2iQevB5oiL3Pwm      BolognaPhysicalTwin
8OV7UspaBE6xjnRE              DigitalRoad
TWdtGEuBbwgP7K0m       DigitalRoadSegment
gdeDA0W3USyZ4W4h      DigitalTrafficLight
NN2USQVIGNXbJuFz       DigitalTrafficLoop
LH4fCkdjthDRwsMC             DigitalModel
qdIoz72JoQ023tFc      DigitalModelManager
uvl9NXse9JgcYndH               RoadShadow
Ti9uDdS6xUv3ScaW        TrafficLoopShadow
Z2XQx1nv8LmBXdre         DigitalDataTrace
Cjrq6N0dtBjCxwJR            DigitalShadow
HLoECxUOVV3e9nNJ     DigitalShadowManager
Ss4juLPEcHnmYCjB       DigitalTwinManager
525aOFCjMOZdirbi    DigitalRepresentation
L6sbK9gAXeunt1mM  TrafficLoopDataProvider
BrHGue0r9yPlMeJB               P2DAdapter
ckbqWnFnU33pqjmj                  Adapter
ddDwSBOXW3Q4LdMp TrafficLightDataReceiver
V9ajJ3yTb1RSkKSS               D2PAdapter
bXVTTFlRnAJ0R8bx           ServiceManager
1RqCkhZ2XxYoPbof     FeedbackTrafficLight
xCdjjqz4UGaFITIA           

### **PIM Generation: XML**

In [109]:
def generate_xml(pimClasses, pimRelations, output_file="generated_project.xml"):
    # Create the root element <Project>
    project = ET.Element("Project", {
        "Author": "Generated",
        "Name": "GeneratedMDADT",
        "UmlVersion": "2.x",
        "Xml_structure": "simple"
    })

    # Create the <Models> container under <Project>
    models = ET.SubElement(project, "Models")

    # Add all classes from pimClasses DataFrame
    for _, row in pimClasses.iterrows():
        # Create <Class> element for each row
        class_element = ET.SubElement(models, "Class", {
            "Id": row["Class ID"],
            "Name": row["Class Name"]
        })

    # Add all relationships from pimRelations DataFrame
    relationship_container = ET.SubElement(models, "ModelRelationshipContainer", {
        "Id": "Relationships",
        "Name": "relationships"
    })

    for _, row in pimRelations.iterrows():
        relationship_type = row["Relationship Type"]
        from_class_id = row["From Class ID"]
        to_class_id = row["To Class ID"]
        aggregation = row.get("Aggregation", None)

        if relationship_type == "Generalization":
            # Add <Generalization> element
            ET.SubElement(relationship_container, "Generalization", {
                "From": from_class_id,
                "To": to_class_id
            })
        elif relationship_type == "Aggregation":
            # Add <Association> element with AggregationKind for aggregation
            association = ET.SubElement(relationship_container, "Association", {
                "From": from_class_id,
                "To": to_class_id
            })
            ET.SubElement(association, "FromEnd", {
                "AggregationKind": aggregation if aggregation else "None"
            })
            ET.SubElement(association, "ToEnd", {
                "AggregationKind": "None"
            })
        elif relationship_type == "Usage":
            # Add <Usage> element
            ET.SubElement(relationship_container, "Usage", {
                "From": from_class_id,
                "To": to_class_id
            })
        elif relationship_type == "CompliantWith":
            # Add <Association> for CompliantWith relation
            compliant_with = ET.SubElement(relationship_container, "Association", {
                "From": from_class_id,
                "To": to_class_id,
                "Relationship Type": "CompliantWith"
            })

    # Write the XML tree to a file
    tree = ET.ElementTree(project)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"XML file '{output_file}' generated successfully.")

In [ ]:
generate_xml(pimClasses, pimRelations)

In [117]:
def generateTimestamp():
    return datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]

def writeXML(project, XMLPath):
    tree = ET.ElementTree(project)
    with open(XMLPath, "wb") as file:
        tree.write(file, encoding="UTF-8", xml_declaration=True)
        print(f"XML file generated successfully in '{XMLPath}' path.")

def prettify(element):
    """Return a pretty-printed XML string for the Element."""
    rough_string = ET.tostring(element, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    reparsed = reparsed.toprettyxml(indent="    ")
    #prettyXML =  "\n".join(line for line in reparsed.split("\n") if line.strip())
    return reparsed

In [218]:
def addStaticProjectOptions(projectInfo):
    projectOptions = ET.SubElement(projectInfo, "ProjectOptions")
    diagramOptions = ET.SubElement(projectOptions, "DiagramOptions", 
    ActivityDiagramControlFlowDisplayOption="\\u0000", 
    ActivityDiagramShowActionCallBehaviorOption="\\u0000", 
    ActivityDiagramShowActivityEdgeWeight="true", 
    ActivityDiagramShowObjectNodeType="true", 
    ActivityDiagramShowPartitionHandle="\\u0000", 
    AddDataStoresExtEntitiesToDecomposedDFD="\\u0002", 
    AlignColumnProperties="true", 
    AllowConfigShowInOutFlowButtonsInDataFlowDiagram="false", 
    AutoGenerateRoleName="false", 
    AutoSetAttributeType="true", 
    AutoSetColumnType="true", 
    AutoSyncRoleName="true", 
    BpdAutoStretchPools="true", 
    BpdConnectGatewayWithFlowObjectInDifferentPool="\\u0000", 
    BpdDefaultConnectionPointStyle="\\u0001", 
    BpdDefaultConnectorStyle="\\u0005", 
    BpdDhowIdOption="\\u0000", 
    BpdShowActivitiesTypeIcon="true", 
    BusinessProcessDiagramDefaultLanguage="English", 
    ClassVisibilityStyle="\\u0001", 
    ConnectorLabelOrientation="\\u0000", 
    CreateOneMessagePerDirection="true", 
    DecisionMergeNodeConnectionPointStyle="\\u0000", 
    DefaultAssociationEndNavigable="\\u0001", 
    DefaultAssociationEndVisibility="\\u0000", 
    DefaultAssociationShowFromMultiplicity="true", 
    DefaultAssociationShowFromRoleName="true", 
    DefaultAssociationShowFromRoleVisibility="true", 
    DefaultAssociationShowStereotypes="true", 
    DefaultAssociationShowToMultiplicity="true", 
    DefaultAssociationShowToRoleName="true", 
    DefaultAssociationShowToRoleVisibility="true", 
    DefaultAttributeMultiplicity="false", 
    DefaultAttributeType="", 
    DefaultAttributeVisibility="\\u0001", 
    DefaultClassAttributeMultiplicity="", 
    DefaultClassAttributeMultiplicityOrdered="false", 
    DefaultClassAttributeMultiplicityUnique="true", 
    DefaultClassInterfaceBall="false", 
    DefaultClassVisibility="\\u0004", 
    DefaultColumnType="integer(10)", 
    DefaultConnectionPointStyle="\\u0000", 
    DefaultConnectorStyle="\\u0001", 
    DefaultDiagramBackground="rgb(255, 255, 255)", 
    DefaultDisplayAsRobustnessAnalysisIcon="true", 
    DefaultDisplayAsRobustnessAnalysisIconInSequenceDiagram="true", 
    DefaultDisplayAsStereotypeIcon="false", 
    DefaultFontColor="rgb(0, 0, 0)", 
    DefaultGenDiagramTypeFromScenario="\\u0000", 
    DefaultHtmlDocFontColor="rgb(0, 0, 0)", 
    DefaultLineJumps="\\u0000", 
    DefaultOperationVisibility="\\u0004", 
    DefaultParameterDirection="\\u0002", 
    DefaultShowAttributeInitialValue="true", 
    DefaultShowAttributeOption="\\u0001", 
    DefaultShowClassMemberStereotype="true", 
    DefaultShowDirection="false", 
    DefaultShowMultiplicityConstraints="false", 
    DefaultShowOperationOption="\\u0001", 
    DefaultShowOperationSignature="true", 
    DefaultShowOrderedMultiplicityConstraint="true", 
    DefaultShowOwnedAssociationEndAsAttribute="true", 
    DefaultShowOwner="false", 
    DefaultShowOwnerSkipModelInFullyQualifiedOwnerSignature="true", 
    DefaultShowReceptionOption="\\u0001", 
    DefaultShowTemplateParameter="true", 
    DefaultShowTypeOption="\\u0001", 
    DefaultShowUniqueMultiplicityConstraint="true", 
    DefaultTypeOfSubProcess="\\u0000", 
    DefaultWrapClassMember="false", 
    DrawTextAnnotationOpenRectangleFollowConnectorEnd="true", 
    EnableMinimumSize="true", 
    EntityColumnConstraintsPresentation="\\u0002", 
    ErdIndexNumOfDigits="-1", 
    ErdIndexPattern="{table_name}", 
    ErdIndexPatternSyncAutomatically="true", 
    ErdManyToManyJoinTableDelimiter="_", 
    EtlTableDiagramFontSize="14", 
    ExpandedSubProcessDiagramContent="\\u0001", 
    ForeignKeyArrowHeadSize="\\u0002", 
    ForeignKeyConnectorEndPointAssociatedColumn="false", 
    ForeignKeyNamePattern="{reference_table_name}{reference_column_name}", 
    ForeignKeyNamePatternCaseHandling="0", 
    ForeignKeyRelationshipPattern="{association_name}", 
    FractionalMetrics="true", 
    GeneralizationSetNotation="\\u0001", 
    GraphicAntiAliasing="true", 
    GridDiagramFontSize="14", 
    LineJumpSize="\\u0000", 
    ModelElementNameAlignment="\\u0004", 
    MultipleLineClassName="\\u0001", 
    PaintConnectorThroughLabel="false", 
    PointConnectorEndToCompartmentMember="true", 
    PrimaryKeyConstraintPattern="", 
    PrimaryKeyNamePattern="ID", 
    RenameConstructorAfterRenameClass="\\u0000", 
    RenameExtensionPointToFollowExtendUseCase="\\u0000", 
    ShapeAutoFitSize="false", 
    ShowActivationsInSequenceDiagram="true", 
    ShowActivityStateNodeCaption="524287", 
    ShowArtifactOption="\\u0002", 
    ShowAssociatedDiagramNameOfInteraction="false", 
    ShowAssociationRoleStereotypes="true", 
    ShowAttributeGetterSetter="false", 
    ShowBSElementCode="true", 
    ShowClassEmptyCompartments="false", 
    ShowColumnDefaultValue="false", 
    ShowColumnNullable="true", 
    ShowColumnType="true", 
    ShowColumnUniqueConstraintName="false", 
    ShowColumnUserType="false", 
    ShowComponentOption="\\u0002", 
    ShowExtraColumnProperties="true", 
    ShowInOutFlowButtonsInDataFlowDiagram="false", 
    ShowInOutFlowsInSubLevelDiagram="true", 
    ShowMessageOperationSignatureForSequenceAndCommunicationDiagram="true", 
    ShowMessageStereotypeInSequenceAndCommunicationDiagram="true", 
    ShowNumberInCollaborationDiagram="true", 
    ShowNumberInSequenceDiagram="true", 
    ShowPackageNameStyle="\\u0000", 
    ShowParameterNameInOperationSignature="true", 
    ShowRowGridLineWithinCompClassDiagram="false", 
    ShowRowGridLineWithinCompERD="true", 
    ShowRowGridLineWithinORMDiagram="true", 
    ShowSchemaNameInERD="true", 
    ShowTransitionTrigger="\\u0000", 
    ShowUseCaseExtensionPoint="true", 
    ShowUseCaseID="false", 
    SnapConnectorsAfterZoom="false", 
    StateShowParametersOfInternalActivities="false", 
    StateShowPrePostConditionAndBodyOfInternalActivities="true", 
    StopTargetLifelineOnCreateDestroyMessage="\\u0002", 
    SupportHtmlTaggedValue="false", 
    SupportMultipleLineAttribute="true", 
    SuppressImpliedMultiplicityForAttributeAssociationEnd="false", 
    SyncAssociationNameWithAssociationClass="\\u0000", 
    SyncAssociationRoleNameWithReferencedAttributeName="true", 
    SyncDocOfInterfaceToSubClass="\\u0000", 
    TextAntiAliasing="true", 
    TextualAnalysisGenerateRequirementTextOption="\\u0001", 
    TextualAnalysisHighlightOption="\\u0000", 
    UnnamedIndexPattern="{table_name}_{column_name}", 
    UseStateNameTab="false", 
    WireflowDiagramDevice="0", 
    WireflowDiagramShowActiveFlowLabel="true", 
    WireflowDiagramTheme="0", 
    WireflowDiagramWireflowShowPreview="true", 
    WireflowDiagramWireflowShowScreenId="true"
    )

# Add additional sub-elements such as GeneralOptions, InstantReverseOptions, etc.
    ET.SubElement(projectOptions, "GeneralOptions", ConfirmSubLevelIdWithDot="true", QuickAddGlossaryTermParentModelId="default")
    ET.SubElement(projectOptions, "InstantReverseOptions", CalculateGeneralizationAndRealization="false", CreateShapeForParentModelOfDraggedClassPackage="false", ReverseGetterSetter="\\u0000", ReverseOperationImplementation="false", ShowPackageForNewDiagram="\\u0001", ShowPackageOwner="\\u0000")
    ET.SubElement(projectOptions, "ModelQualityOptions", EnableModelQualityChecking="false")
    ormOptions = ET.SubElement(projectOptions, "ORMOptions", 
    DecimalPrecision="19", 
    DecimalScale="0", 
    ExportCommentToDatabase="true", 
    FormattedSQL="false", 
    GenerateAssociationWithAttribute="false", 
    GenerateDiagramFromORMWizards="true", 
    GetterSetterVisibility="\\u0000", 
    IdGeneratorType="native", 
    MappingFileColumnOrder="\\u0000", 
    NumericToClassType="\\u0000", 
    QuoteSQLIdentifier="\\u0000", 
    RecreateShapeWhenSync="false", 
    SyncToClassDiagramAttributeName="\\u0001", 
    SyncToClassDiagramAttributeNamePrefix="", 
    SyncToClassDiagramAttributeNameSuffix="", 
    SyncToClassDiagramClassName="\\u0000", 
    SyncToClassDiagramClassNamePrefix="", 
    SyncToClassDiagramClassNameSuffix="", 
    SyncToERDColumnName="\\u0000", 
    SyncToERDColumnNamePrefix="", 
    SyncToERDColumnNameSuffix="", 
    SyncToERDTableName="\\u0004", 
    SyncToERDTableNamePrefix="", 
    SyncToERDTableNameSuffix="", 
    SynchronizeDefaultValueToColumn="false", 
    SynchronizeName="\\u0002", 
    TablePerSubclassFKMapping="\\u0000", 
    UpperCaseSQL="true", 
    UseDefaultDecimal="true", 
    WrappingServletRequest="\\u0001")

    # Add requirementDiagramOptions in camelCase
    requirementDiagramOptions = ET.SubElement(projectOptions, "RequirementDiagramOptions", 
        DefaultWrapMember="true", 
        ShowAttributes="\\u0001", 
        SupportHTMLAttribute="false")

    # Add stateCodeEngineOptions in camelCase
    stateCodeEngineOptions = ET.SubElement(projectOptions, "StateCodeEngineOptions", 
        AutoCreateInitialStateInStateDiagram="true", 
        AutoCreateTransitionMethods="true", 
        DefaultInitialStateLocationX="-1", 
        DefaultInitialStateLocationY="-1", 
        GenerateDebugMessage="false", 
        GenerateSample="true", 
        GenerateTryCatch="true", 
        Language="\\u0000", 
        RegenerateTransitionMethods="false", 
        SyncTransitionMethods="true")

    # Add warningOptions in camelCase
    warningOptions = ET.SubElement(projectOptions, "WarningOptions", CreateORMClassInDefaultPackage="true")

    # Add poRepository and its children in camelCase
    poRepository = ET.SubElement(projectOptions, "PORepository")
    poUserIdFormats = ET.SubElement(poRepository, "POUserIDFormats")

    # Add poUserIdFormats child elements in camelCase
    ET.SubElement(poUserIdFormats, "POUserIDFormat", 
        Digits="2", 
        Guid="false", 
        Id="RBeZzEmGAqAC8QlN", 
        LastNumericValue="0", 
        ModelType="BPMNElement", 
        Prefix="BP", 
        Suffix="")

    ET.SubElement(poUserIdFormats, "POUserIDFormat", 
        Digits="2", 
        Guid="false", 
        Id="RBeZzEmGAqAC8QlO", 
        LastNumericValue="0", 
        ModelType="Actor", 
        Prefix="AC", 
        Suffix="")

    ET.SubElement(poUserIdFormats, "POUserIDFormat", 
        Digits="3", 
        Guid="false", 
        Id="RBeZzEmGAqAC8QlQ", 
        LastNumericValue="0", 
        ModelType="Requirement", 
        Prefix="REQ", 
        Suffix="")

    ET.SubElement(poUserIdFormats, "POUserIDFormat", 
        Digits="3", 
        Guid="false", 
        Id="RBeZzEmGAqAC8QlR", 
        LastNumericValue="0", 
        ModelType="BusinessRule", 
        Prefix="BR", 
        Suffix="")

    ET.SubElement(poUserIdFormats, "POUserIDFormat", 
        Digits="-1", 
        Guid="false", 
        Id="RBeZzEmGAqAC8QlS", 
        LastNumericValue="0", 
        ModelType="BusinessProcessDiagram", 
        Prefix="", 
        Suffix="")

def addStaticDataType(models, projectAuthor):
    data_types = [
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "k2eZzEmGAqAC8QXD", "Name": "boolean", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXE", "Name": "byte", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXF", "Name": "char", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXG", "Name": "double", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXH", "Name": "float", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXI", "Name": "int", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXJ", "Name": "long", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXK", "Name": "short", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXL", "Name": "void", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"},
         
        {"BacklogActivityId": "0", "Documentation_plain": "", "Id": "G2eZzEmGAqAC8QXM", "Name": "string", 
         "PmAuthor": projectAuthor, "PmCreateDateTime": generateTimestamp(), 
         "PmLastModified": generateTimestamp(), "QualityReason_IsNull": "true", 
         "QualityScore": "-1", "UserIDLastNumericValue": "0", "UserID_IsNull": "true"}
        ]
    for data in data_types:
        ET.SubElement(models, "DataType", **data)


In [227]:
def addUsageStereotype(projectAuthor, models, existingIDs, idLength):
    usageStereotypeIdRef = generateId(existingIDs, idLength)
    existingIDs.add(usageStereotypeIdRef)
    ET.SubElement(models, 'Stereotype', {
    'Abstract': 'false',
    'BacklogActivityId': '0',
    'BaseType': 'Usage',
    'Documentation_plain': '',
    'IconPath_IsNull': 'true',
    'Id': usageStereotypeIdRef,
    'Leaf': 'false',
    'Name': 'use',
    'PmAuthor': projectAuthor,
    'PmCreateDateTime': generateTimestamp(),
    'PmLastModified': generateTimestamp(),
    'QualityReason_IsNull': 'true',
    'QualityScore': '-1',
    'Root': 'false',
    'UserIDLastNumericValue': '0',
    'UserID_IsNull': 'true'
    })
    return usageStereotypeIdRef
    
def addUsageRelation(projectAuthor, modelChildrenMCC, fromID, toID, usageShapeID, usageMasterIdRef, usageStereotypeIdRef):
    """
    Function to add a usage relation with sub-elements: Stereotypes and MasterView.
    
    Args:
        projectAuthor: The author of the project.
        modelChildrenContainer: Parent XML element under which the 'Usage' element will be added.
        fromID: The 'From' attribute for the 'Usage' element.
        toID: The 'To' attribute for the 'Usage' element.
        usageShapeID: The unique ID for the 'Usage' element.
        usageMasterIdRef: The MasterView reference ID.
        usageStereotypeIdRef: The Stereotype reference ID.
    """
    # Create the Usage element with the necessary attributes
    usage = ET.SubElement(modelChildrenMCC, 'Usage', {
        'BacklogActivityId': '0',
        'Documentation_plain': '',
        'From': fromID,
        'Id': usageShapeID,
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
        'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'To': toID,
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })
    
    # Add the Stereotypes sub-element
    stereotypes = ET.SubElement(usage, 'Stereotypes')
    ET.SubElement(stereotypes, 'Stereotype', {
        'Idref': usageStereotypeIdRef,  # Provided stereotype ID reference
        'Name': 'use'                   # Fixed name for stereotype
    })
    
    # Add the MasterView sub-element
    master_view = ET.SubElement(usage, 'MasterView')
    ET.SubElement(master_view, 'Usage', {
        'Idref': usageMasterIdRef  # MasterView reference ID
    })

    return modelChildrenMCC

def addUsageRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, usageStereotypeIdRef, existingIDs, idLength):
    """
    Function to add a container of Usage relations based on the pimRelations DataFrame.
    
    Args:
        projectAuthor: The author of the project.
        modelChildren: Parent XML element under which the Usage elements will be added.
        pimRelations: DataFrame containing the relationship data.
        pimClasses: DataFrame or list containing the class information.
    """
    
    # Variable to track if the ModelRelationshipContainer has been created
    modelChildrenContainer = None
    modelChildrenMCC = None
    
    
    # Iterate through each row in the pimRelations DataFrame
    for index, row in pimRelations.iterrows():
        relationType = row["Relationship Type"]
        
        if relationType == 'Usage':  # Check if the relation is of type 'Usage'

            # Create the ModelRelationshipContainer only once, before adding any Usage relations
            if modelChildrenContainer is None:
                # Generate the container ID
                IDContainer = generateId(existingIDs, idLength)
                # Add the ModelRelationshipContainer to the modelChildren
                modelChildrenContainer = ET.SubElement(modelChildren, 'ModelRelationshipContainer', 
                                            BacklogActivityId="0", 
                                            Documentation_plain="", 
                                            Id=IDContainer, 
                                            Name="Usage", 
                                            PmAuthor=projectAuthor, 
                                            PmCreateDateTime=generateTimestamp(), 
                                            PmLastModified=generateTimestamp(), 
                                            QualityReason_IsNull="true", 
                                            QualityScore="-1", 
                                            UserIDLastNumericValue="0", 
                                            UserID_IsNull="true")
                modelChildrenMCC = ET.SubElement(modelChildrenContainer, 'ModelChildren')
            
            fromClassID = row["From Class ID"]
            toClassID = row["To Class ID"]
            
            # Generate a unique Shape ID
            usageShapeID = generateId(existingIDs, idLength)
            # Add the new Shape ID to the pimRelations DataFrame
            pimRelations.at[index, 'ShapeID'] = usageShapeID
            existingIDs.add(usageShapeID)
            
            # Generate a unique MasterView reference ID
            usageMasterIdRef = generateId(existingIDs, idLength)
            # Add the new Master ID to the pimRelations DataFrame
            pimRelations.at[index, 'MasterID'] = usageMasterIdRef
            existingIDs.add(usageMasterIdRef)
            pimRelations.at[index, 'StereotypeID'] = usageStereotypeIdRef 
            
            # Add the usage relation to the XML
            modelChildrenMCC = addUsageRelation(projectAuthor, modelChildrenMCC, fromClassID, toClassID, usageShapeID, usageMasterIdRef, usageStereotypeIdRef)
    
    return modelChildren

def addAssociationRelation(projectAuthor, modelChildrenMCC, fromID, fromName, toID, toName, aggregationKind, fromEndAssociationID, fromEndQualifierID, 
                           toEndAssociationID, toEndQualifierID, associationMasterIdRef, associationShapeID):
    """
    Function to add an association/aggregation/composition relation with sub-elements: FromEnd, ToEnd, and MasterView.
    
    Args:
        projectAuthor: The author of the project.
        modelChildrenContainer: Parent XML element under which the 'Association' element will be added.
        fromID: The 'From' attribute for the 'Association' element.
        fromName: The name of the 'From' class.
        toID: The 'To' attribute for the 'Association' element.
        toName: The name of the 'To' class.
        aggregationKind: The type of aggregation ("None", "Shared", "Composite").
        fromEndAssociationID: The unique ID for the 'FromEnd' association element.
        fromEndQualifierID: The unique ID for the 'FromEnd' qualifier element.
        toEndAssociationID: The unique ID for the 'ToEnd' association element.
        toEndQualifierID: The unique ID for the 'ToEnd' qualifier element.
        associationMasterIdRef: The MasterView reference ID.
        associationShapeID: The unique ID for the 'Association' element.
    """
    # Create the Association element with the necessary attributes
    association = ET.SubElement(modelChildrenMCC, 'Association', {
        'Abstract': 'false',
        'BacklogActivityId': '0',
        'Derived': 'false',
        'Direction': 'From To',
        'Documentation_plain': '',
        'EndRelationshipFromMetaModelElement': fromID,
        'EndRelationshipToMetaModelElement': toID,
        'Id': associationShapeID,
        'Leaf': 'false',
        'OrderingInProfile': '-1',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
        'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })
    
    # Add the FromEnd sub-element with the specified aggregationKind
    fromEnd = ET.SubElement(association, 'FromEnd')
    fromAssociationEnd = ET.SubElement(fromEnd, 'AssociationEnd', {
        'AggregationKind': aggregationKind,
        'BacklogActivityId': '0',
        'ConnectToCodeModel': '1',
        'DefaultValue_IsNull': 'true',
        'Derived': 'false',
        'DerivedUnion': 'false',
        'Documentation_plain': '',
        'EndModelElement': fromID,
        'Id': fromEndAssociationID,
        'JavaCodeAttributeName': '',
        'Leaf': 'false',
        'Multiplicity': 'Unspecified',
        'Navigable': 'Navigable',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),
        'PmLastModified_IsNull': 'true',
        'ProvidePropertyGetterMethod': 'false',
        'ProvidePropertySetterMethod': 'false',
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'ReadOnly': 'false',
        'Static': 'false',
        'TypeModifier': '',
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })
    # Add the Qualifier for the FromEnd
    qualifierEnd = ET.SubElement(fromAssociationEnd, 'Qualifier', {
        'BacklogActivityId': '0',
        'Documentation_plain': '',
        'Id': fromEndQualifierID,
        'Name': '',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),
        'PmLastModified_IsNull': 'true',
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
    })

    # Add the Type for the FromEnd <Class Idref="fromID" Name="fromName" />
    typeEnd = ET.SubElement(fromAssociationEnd, 'Type')
    ET.SubElement(typeEnd, 'Class', {
        'Idref': fromID,
        'Name': fromName
    })

    # Add the ToEnd sub-element (with no aggregation, as per example)
    toEnd = ET.SubElement(association, 'ToEnd')
    toAssociationEnd = ET.SubElement(toEnd, 'AssociationEnd', {
        'AggregationKind': 'None',
        'BacklogActivityId': '0',
        'ConnectToCodeModel': '1',
        'DefaultValue_IsNull': 'true',
        'Derived': 'false',
        'DerivedUnion': 'false',
        'Documentation_plain': '',
        'EndModelElement': toID,
        'Id': toEndAssociationID,
        'JavaCodeAttributeName': '',
        'Leaf': 'false',
        'Multiplicity': 'Unspecified',
        'Navigable': 'Navigable',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),
        'PmLastModified_IsNull': 'true',
        'ProvidePropertyGetterMethod': 'false',
        'ProvidePropertySetterMethod': 'false',
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'ReadOnly': 'false',
        'Static': 'false',
        'TypeModifier': '',
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })
    
    # Add the Qualifier for the ToEnd
    qualifierTo = ET.SubElement(toAssociationEnd, 'Qualifier', {
        'BacklogActivityId': '0',
        'Documentation_plain': '',
        'Id': toEndQualifierID,
        'Name': '',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),
        'PmLastModified_IsNull': 'true',
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
    })
    
    # Add the Type for the ToEnd <Class Idref="toID" Name="toName" />
    typeTo = ET.SubElement(toAssociationEnd, 'Type')
    ET.SubElement(typeTo, 'Class', {
        'Idref': toID,
        'Name': toName
    })

    # Add the MasterView sub-element
    masterView = ET.SubElement(association, 'MasterView')
    ET.SubElement(masterView, 'Association', {
        'Idref': associationMasterIdRef  # MasterView reference ID
    })

    return modelChildrenMCC

def addAssociationRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, idLength):
    """
    Function to add a container of Association/Aggregation/Composition relations based on the pimRelations DataFrame.
    
    Args:
        projectAuthor: The author of the project.
        modelChildren: Parent XML element under which the Association elements will be added.
        pimRelations: DataFrame containing the relationship data.
        pimClasses: DataFrame or list containing the class information.
    """
    
    # Variable to track if the ModelRelationshipContainer has been created
    modelChildrenContainer = None
    modelChildrenMCC = None
    # Iterate through each row in the pimRelations DataFrame
    for index, row in pimRelations.iterrows():
        relationType = row["Relationship Type"]
        
        # Check for association, aggregation (shared), or composition (composite)
        if relationType in ['Association', 'Aggregation', 'Composition']:  

            # Create the ModelRelationshipContainer only once, before adding any association relations
            if modelChildrenContainer is None:
                # Generate the container ID
                IDContainer = generateId(existingIDs, idLength)
                # Add the ModelRelationshipContainer to the modelChildren
                modelChildrenContainer = ET.SubElement(modelChildren, 'ModelRelationshipContainer', 
                                            BacklogActivityId="0", 
                                            Documentation_plain="", 
                                            Id=IDContainer, 
                                            Name="Association", 
                                            PmAuthor=projectAuthor, 
                                            PmCreateDateTime=generateTimestamp(), 
                                            PmLastModified=generateTimestamp(), 
                                            QualityReason_IsNull="true", 
                                            QualityScore="-1", 
                                            UserIDLastNumericValue="0", 
                                            UserID_IsNull="true")
                modelChildrenMCC = ET.SubElement(modelChildrenContainer, 'ModelChildren')
            
            fromClassID = row["From Class ID"]
            toClassID = row["To Class ID"]
            fromClassName = row["From Class Name"]
            toClassName = row["To Class Name"]
            
            # Generate a unique Shape ID
            associationShapeID = generateId(existingIDs, idLength)
            # Add the new Shape ID to the pimRelations DataFrame
            pimRelations.at[index, 'ShapeID'] = associationShapeID
            existingIDs.add(associationShapeID)
            
            # Generate a unique MasterView reference ID
            associationMasterIdRef = generateId(existingIDs, idLength)
            # Add the new Master ID to the pimRelations DataFrame
            pimRelations.at[index, 'MasterID'] = associationMasterIdRef
            existingIDs.add(associationMasterIdRef)
            
            # Generate IDs for Association ends and Qualifiers
            fromEndAssociationID = generateId(existingIDs, idLength)
            existingIDs.add(fromEndAssociationID)
            toEndAssociationID = generateId(existingIDs, idLength)
            existingIDs.add(toEndAssociationID)
            fromEndQualifierID = generateId(existingIDs, idLength)
            existingIDs.add(fromEndQualifierID)
            toEndQualifierID = generateId(existingIDs, idLength)
            existingIDs.add(toEndQualifierID)

            # Determine the type of aggregation for the AssociationEnd (Shared, Composite, None)
            aggregationKind = 'None'
            if relationType == 'Aggregation':
                aggregationKind = 'Shared'
            elif relationType == 'Composition':
                aggregationKind = 'Composite'
            
            # Add the association relation to the XML
            modelChildrenMCC = addAssociationRelation(
                projectAuthor, 
                modelChildrenMCC, 
                fromClassID, 
                fromClassName, 
                toClassID, 
                toClassName, 
                aggregationKind, 
                fromEndAssociationID, 
                fromEndQualifierID, 
                toEndAssociationID, 
                toEndQualifierID, 
                associationMasterIdRef, 
                associationShapeID
            )
    
    return modelChildren

def addGeneralizationRelation(projectAuthor, modelChildrenMCC, fromID, fromName, toID, toName, generalizationShapeID, generalizationMasterIdRef):
    """
    Function to add a generalization relation with sub-elements: Generalization and MasterView.
    
    Args:
        projectAuthor: The author of the project.
        modelChildrenMCC: Parent XML element under which the 'Generalization' element will be added.
        fromID: The 'From' class ID for the 'Generalization' element.
        fromName: The name of the 'From' class.
        toID: The 'To' class ID for the 'Generalization' element.
        toName: The name of the 'To' class.
        generalizationShapeID: The unique ID for the 'Generalization' element.
        generalizationMasterIdRef: The MasterView reference ID.
    """
    # Create the Generalization element with the necessary attributes
    
    generalization = ET.SubElement(modelChildrenMCC, 'Generalization', {
        'BacklogActivityId': '0',
        'ConnectToCodeModel': '1',
        'Documentation_plain': '',
        'From': fromID,
        'Id': generalizationShapeID,
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
        'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'Substitutable': 'false',
        'To': toID,
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })

    # Add the MasterView sub-element
    masterView = ET.SubElement(generalization, 'MasterView')
    ET.SubElement(masterView, 'Generalization', {
        'Idref': generalizationMasterIdRef  # MasterView reference ID
    })

    return modelChildrenMCC

def addGeneralizationRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, idLength):
    """
    Function to add a container of Generalization relations based on the pimRelations DataFrame.
    
    Args:
        projectAuthor: The author of the project.
        modelChildren: Parent XML element under which the Generalization elements will be added.
        pimRelations: DataFrame containing the relationship data.
        pimClasses: DataFrame or list containing the class information.
    """
    
    # Variable to track if the ModelRelationshipContainer has been created
    modelChildrenContainer = None
    modelChildrenMCC = None

    
    # Iterate through each row in the pimRelations DataFrame
    for index, row in pimRelations.iterrows():
        relationType = row["Relationship Type"]
        
        # Check for generalization relations
        if relationType == 'Generalization':  

            # Create the ModelRelationshipContainer only once, before adding any generalization relations
            if modelChildrenContainer is None:
                # Generate the container ID
                IDContainer = generateId(existingIDs, idLength)
                # Add the ModelRelationshipContainer to the modelChildren
                modelChildrenContainer = ET.SubElement(modelChildren, 'ModelRelationshipContainer', 
                                            BacklogActivityId="0", 
                                            Documentation_plain="", 
                                            Id=IDContainer, 
                                            Name="Generalization", 
                                            PmAuthor=projectAuthor, 
                                            PmCreateDateTime=generateTimestamp(), 
                                            PmLastModified=generateTimestamp(), 
                                            QualityReason_IsNull="true", 
                                            QualityScore="-1", 
                                            UserIDLastNumericValue="0", 
                                            UserID_IsNull="true")
                modelChildrenMCC = ET.SubElement(modelChildrenContainer, 'ModelChildren')
            
            fromClassID = row["From Class ID"]
            toClassID = row["To Class ID"]
            fromClassName = row["From Class Name"]
            toClassName = row["To Class Name"]
            
            # Generate a unique Shape ID for the generalization
            generalizationShapeID = generateId(existingIDs, idLength)
            # Add the new Shape ID to the pimRelations DataFrame
            pimRelations.at[index, 'ShapeID'] = generalizationShapeID
            existingIDs.add(generalizationShapeID)
            
            # Generate a unique MasterView reference ID
            generalizationMasterIdRef = generateId(existingIDs, idLength)
            # Add the new Master ID to the pimRelations DataFrame
            pimRelations.at[index, 'MasterID'] = generalizationMasterIdRef
            existingIDs.add(generalizationMasterIdRef)

            # Add the generalization relation to the XML
            modelChildrenMCC = addGeneralizationRelation(
                projectAuthor, 
                modelChildrenMCC, 
                fromClassID, 
                fromClassName, 
                toClassID, 
                toClassName, 
                generalizationShapeID, 
                generalizationMasterIdRef
            )
    
    return modelChildren

def addDependencyRelation(projectAuthor, modelChildrenMCC, fromID, toID, dependencyShapeID, dependencyMasterIdRef):
    """
    Function to add a dependency relation with sub-elements: Dependency and MasterView.
    
    Args:
        projectAuthor: The author of the project.
        modelChildrenMCC: Parent XML element under which the 'Dependency' element will be added.
        fromID: The 'From' attribute for the 'Dependency' element.
        toID: The 'To' attribute for the 'Dependency' element.
        dependencyShapeID: The unique ID for the 'Dependency' element.
        dependencyMasterIdRef: The MasterView reference ID.
    """
    # Create the Dependency element with the necessary attributes
    dependency = ET.SubElement(modelChildrenMCC, 'Dependency', {
        'BacklogActivityId': '0',
        'Documentation_plain': '',
        'From': fromID,
        'Id': dependencyShapeID,
        'Name': '&lt;&lt;compliant with&gt;&gt;',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
        'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
        'QualityReason_IsNull': 'true',
        'QualityScore': '-1',
        'To': toID,
        'UserIDLastNumericValue': '0',
        'UserID_IsNull': 'true',
        'Visibility': 'Unspecified'
    })

    # Add the MasterView sub-element
    masterView = ET.SubElement(dependency, 'MasterView')
    ET.SubElement(masterView, 'Dependency', {
        'Idref': dependencyMasterIdRef,
        'Name': '&lt;&lt;compliant with&gt;&gt;'
    })

    return modelChildrenMCC

def addDependencyRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, idLength):
    """
    Function to add a container of Dependency (CompliantWith) relations based on the pimRelations DataFrame.
    
    Args:
        projectAuthor: The author of the project.
        modelChildren: Parent XML element under which the Dependency elements will be added.
        pimRelations: DataFrame containing the relationship data.
        pimClasses: DataFrame or list containing the class information.
    """   
    
    # Variable to track if the ModelRelationshipContainer has been created
    modelChildrenContainer = None
    modelChildrenMCC = None
    

    
    
    # Iterate through each row in the pimRelations DataFrame
    for index, row in pimRelations.iterrows():
        relationType = row["Relationship Type"]
        
        # Check for compliant with relations
        if relationType == 'CompliantWith':  

            # Create the ModelRelationshipContainer only once, before adding any dependency relations
            if modelChildrenContainer is None:
                # Generate the container ID
                IDContainer = generateId(existingIDs, idLength)
                # Add the ModelRelationshipContainer to the modelChildren
                modelChildrenContainer = ET.SubElement(modelChildren, 'ModelRelationshipContainer', 
                                                       BacklogActivityId="0", 
                                                       Documentation_plain="", 
                                                       Id=IDContainer, 
                                                       Name="Dependency", 
                                                       PmAuthor=projectAuthor, 
                                                       PmCreateDateTime=generateTimestamp(), 
                                                       PmLastModified=generateTimestamp(), 
                                                       QualityReason_IsNull="true", 
                                                       QualityScore="-1", 
                                                       UserIDLastNumericValue="0", 
                                                       UserID_IsNull="true")
                modelChildrenMCC = ET.SubElement(modelChildren, 'ModelChildren') 
            
            fromClassID = row["From Class ID"]
            toClassID = row["To Class ID"]
            
            # Generate a unique Shape ID for the dependency
            dependencyShapeID = generateId(existingIDs, idLength)
            pimRelations.at[index, 'ShapeID'] = dependencyShapeID
            existingIDs.add(dependencyShapeID)
            
            dependencyMasterIdRef = generateId(existingIDs, idLength)
            pimRelations.at[index, 'MasterID'] = dependencyMasterIdRef
            existingIDs.add(dependencyMasterIdRef)

            # Add the dependency relation to the XML
            modelChildrenMCC = addDependencyRelation(
                projectAuthor, 
                modelChildrenMCC, 
                fromClassID, 
                toClassID, 
                dependencyShapeID, 
                dependencyMasterIdRef
            )
    
    return modelChildren


In [249]:
def addClassElement(models, projectAuthor, pimClasses, pimRelations, existingIDs, IdLength):
    """
    Function to add Class elements based on the pimClasses DataFrame.
    
    Args:
        models: Parent XML element under which the 'Class' elements will be added.
        projectAuthor: The author of the project.
        pimClasses: DataFrame containing the class information.
        pimRelations: DataFrame containing the relationship data.
    """
    # Iterate through each class in pimClasses DataFrame
    for index, row in pimClasses.iterrows():
        classID = row["Class ID"]
        className = row["Class Name"]
        classIdRef = generateId(existingIDs, IdLength)
        existingIDs.add(classIdRef)
        pimClasses.at[index, 'ClassIdRef'] = classIdRef

        # Create the Class element with the necessary attributes
        classElement = ET.SubElement(models, 'Class', {
            'Abstract': 'false',
            'Active': 'false',
            'BacklogActivityId': '0',
            'BusinessKeyMutable': 'true',
            'BusinessModel': 'false',
            'ConnectToCodeModel': '1',
            'Documentation_plain': '',
            'Id': classIdRef,
            'Leaf': 'false',
            'Name': className,
            'PmAuthor': projectAuthor,
            'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
            'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
            'QualityReason_IsNull': 'true',
            'QualityScore': '-1',
            'Root': 'false',
            'UserIDLastNumericValue': '0',
            'UserID_IsNull': 'true',
            'Visibility': 'public'
        })

        # Filter relations from pimRelations that start from this class (FromSimpleRelationships)
        fromRelations = pimRelations[pimRelations['From Class ID'] == classID]

        if not fromRelations.empty:
            fromSimpleRel = ET.SubElement(classElement, 'FromSimpleRelationships')
            for _, relRow in fromRelations.iterrows():
                relType = relRow["Relationship Type"]
                relIDRef = relRow["ShapeID"]  # Assuming ShapeID is stored in pimRelations
                relName = ''  # Can be left empty unless specified

                # Depending on the relationship type, add the corresponding sub-element
                if relType == 'Generalization':
                    ET.SubElement(fromSimpleRel, 'Generalization', {'Idref': relIDRef, 'Name': relName})
                elif relType == 'Usage':
                    relIDRef = relRow["ShapeID"]  # Assuming ShapeID is stored in pimRelations
                    relName = ''  # Can be left empty unless specified
                    ET.SubElement(fromSimpleRel, 'Usage', {'Idref': relIDRef, 'Name': relName})
                elif relType == 'CompliantWith':
                    ET.SubElement(fromSimpleRel, 'Dependency', {'Idref': relIDRef, 'Name': '&lt;&lt;compliant with&gt;&gt;'})
                # Add more conditions if needed for other relationship types

        # Filter relations from pimRelations that point to this class (ToSimpleRelationships)
        toRelations = pimRelations[pimRelations['To Class ID'] == classID]

        if not toRelations.empty:
            toSimpleRel = ET.SubElement(classElement, 'ToSimpleRelationships')
            for _, relRow in toRelations.iterrows():
                relType = relRow["Relationship Type"]
                relIDRef = relRow["ShapeID"]  # Assuming ShapeID is stored in pimRelations
                relName = ''  # Can be left empty unless specified

                if relType == 'Generalization':
                    ET.SubElement(toSimpleRel, 'Generalization', {'Idref': relIDRef, 'Name': relName})

        # Add the MasterView sub-element
        masterView = ET.SubElement(classElement, 'MasterView')
        ET.SubElement(masterView, 'Class', {'Idref': classIdRef, 'Name': className})

    return models

def addClassDiagram(diagrams, projectAuthor, classDiagramID, classDiagramName):
    """
    Function to add a ClassDiagram element to the Diagrams section with all the detailed attributes.
    
    Args:
        diagrams: Parent XML element under which the 'ClassDiagram' element will be added.
        projectAuthor: The author of the project.
    """
    # Create the ClassDiagram element with all the specified attributes
    classDiagram = ET.SubElement(diagrams, 'ClassDiagram', {
        'AlignToGrid': 'false',
        'AutoFitShapesSize': 'false',
        'ClassFitSizeWhenShowHideMember': 'true',
        'ConnectionPointStyle': '0',
        'ConnectorLabelOrientation': '0',
        'ConnectorLineJumps': '0',
        'ConnectorLineJumpsSize': '0',
        'ConnectorModelElementNameAlignment': '4',
        'ConnectorStyle': '1',
        'DiagramBackground': 'rgb(255, 255, 255)',
        'Editable': 'true',
        'FollowDiagramParentElement': 'true',
        'GeneralizationSetNotation': '2',
        'GridColor': 'rgb(192, 192, 192)',
        'GridHeight': '10',
        'GridVisible': 'false',
        'GridWidth': '10',
        'Height': '595',
        'HideConnectorIfFromToIsHidden': '0',
        'HideEmptyTaggedValues': 'false',
        'Id': classDiagramID, 
        'ImageHeight': '0',
        'ImageScale': '1.0',
        'ImageWidth': '0',
        'InitializeDiagramForCreate': 'true',
        'Maximized': 'true',
        'ModelElementNameAlignment': '4',
        'Name':  classDiagramName, 
        'PaintConnectorThroughLabel': '1',
        'PmAuthor': projectAuthor,
        'PmCreateDateTime': generateTimestamp(),  # Generate creation timestamp
        'PmLastModified': generateTimestamp(),    # Generate last modified timestamp
        'PointConnectorEndToCompartmentMember': 'true',
        'QualityScore': '-1',
        'RequestFitSizeWithPromptUser': 'false',
        'RequestValidateSnapToGrid': 'false',
        'ShapePresentationOption': '0',
        'ShowActivityStateNodeCaption': '524287',
        'ShowAllocatedFrom': 'true',
        'ShowAllocatedTo': 'true',
        'ShowAssociationNavigationArrows': '3',
        'ShowAttributeGetterSetter': 'false',
        'ShowAttributesCodeDetails': '2',
        'ShowAttributesPropertyModifiers': '2',
        'ShowAttributesType': '1',
        'ShowClassEmptyCompartments': '2',
        'ShowClassOwner': '2',
        'ShowClassReferencedAttributes': 'true',
        'ShowColorLegend': 'false',
        'ShowConnectorLegend': 'false',
        'ShowConnectorName': '0',
        'ShowConstraints': 'false',
        'ShowDefaultPackage': 'true',
        'ShowEllipsisForUnshownClassMembers': '2',
        'ShowInformationItemOption': '2',
        'ShowOperationsCodeDetails': '2',
        'ShowOperationsParameters': '1',
        'ShowOperationsReturnType': '1',
        'ShowPMAuthor': 'false',
        'ShowPMDifficulty': 'false',
        'ShowPMDiscipline': 'false',
        'ShowPMIteration': 'false',
        'ShowPMPhase': 'false',
        'ShowPMPriority': 'false',
        'ShowPMStatus': 'false',
        'ShowPMVersion': 'false',
        'ShowPackageNameStyle': '0',
        'ShowPackageOwner': '2',
        'ShowParametersCodeDetails': '2',
        'ShowShapeLegend': 'false',
        'ShowShapeStereotypeIconName': 'true',
        'ShowStereotypes': 'true',
        'ShowTaggedValues': 'false',
        'ShowTemplateInfoOfGeneralizationAndRealization': 'false',
        'SuppressImplied1MultiplicityForAttributeAndAssociationEnd': 'false',
        'TeamworkCreateDateTime': '0',
        'TrimmedHeight': '0',
        'TrimmedWidth': '0',
        'Width': '1255',
        'X': '0',
        'Y': '0',
        'ZoomRatio': '0.9',
        "_globalPaletteOption": "true",
        "documentation": """&lt;html&gt;&#13;&#10;&lt;head&gt;&#13;&#10;&lt;style type=&quot;text/css&quot;&gt;&#10;&lt;!--&#10; body { color: #000000; font-family: Dialog; font-size: 12px }&#10;--&gt;&#10;&lt;/style&gt;&#13;&#10;&lt;/head&gt;&#13;&#10;&lt;body&gt;&#13;&#10;&lt;p&gt;&#13;&#10;&lt;/p&gt;&#13;&#10;&lt;/body&gt;&#13;&#10;&lt;/html&gt;"""
    })

    return classDiagram

In [252]:
def generateXYValues(pimClasses, pimRelations):
    """
    Function to generate unique X, Y, and Z values for the classes based on relationships in pimRelations.
    Classes that are connected should be placed near each other, but still have unique positions.
    
    Args:
        pimClasses: DataFrame containing class details.
        pimRelations: DataFrame containing relationships between classes.
        
    Returns:
        Updated pimClasses with X, Y, and Z values.
    """
    # Parameters for the grid spacing
    xSpacing = 200  # Horizontal space between classes
    ySpacing = 100  # Vertical space between rows of classes
    xOffset = 50    # Slight offset to avoid exact overlapping
    yOffset = 50    # Slight offset for Y values for better separation
    
    # Initialize starting positions and ZOrder
    xPos = 100  # Starting X coordinate
    yPos = 100  # Starting Y coordinate
    zOrderValue = 4  # Starting Z-order value
    
    # Track placed classes and their positions
    placedClasses = {}
    
    # Iterate over each class and assign positions
    for index, classRow in pimClasses.iterrows():
        classId = classRow['ClassIdRef']
        
        # Check if the class has already been placed (i.e., already has X, Y values)
        if classId in placedClasses:
            continue  # Skip already placed classes
        
        # Assign the initial position for the class if not already placed
        pimClasses.at[index, 'X'] = xPos
        pimClasses.at[index, 'Y'] = yPos
        pimClasses.at[index, 'ZOrder'] = zOrderValue
        placedClasses[classId] = (xPos, yPos)
        
        # Increment ZOrder for the next class
        zOrderValue += 1
        
        # Fetch all relationships for this class (both as a source and destination)
        relatedClasses = pimRelations[(pimRelations['From Class ID'] == classId) |
                                      (pimRelations['To Class ID'] == classId)]
        
        # For each related class, place it near the current class
        for _, relation in relatedClasses.iterrows():
            relatedClassId = relation['To Class ID'] if relation['From Class ID'] == classId else relation['From Class ID']
            
            # Check if the related class has already been placed
            if relatedClassId in placedClasses:
                continue  # Skip if the related class is already placed
            
            # Slightly adjust X and Y to place the related class nearby but not overlapping
            newXPos = xPos + xOffset
            newYPos = yPos + yOffset
            
            # Find the index of the related class in pimClasses
            relatedClassIndex = pimClasses[pimClasses['ClassIdRef'] == relatedClassId].index[0]
            
            # Set the X, Y, and ZOrder values for the related class
            pimClasses.at[relatedClassIndex, 'X'] = newXPos
            pimClasses.at[relatedClassIndex, 'Y'] = newYPos
            pimClasses.at[relatedClassIndex, 'ZOrder'] = zOrderValue
            
            # Track the new class placement
            placedClasses[relatedClassId] = (newXPos, newYPos)
            
            # Increment ZOrder and X/Y for the next related class
            zOrderValue += 1
            xOffset += xSpacing
            yOffset += ySpacing
        
        # Update X and Y for the next group of unrelated classes (do not reset X/Y)
        xPos += xSpacing
        yPos += ySpacing
    
    return pimClasses

def addClassShapes(shapes, pimClasses, pimRelations): 
    """
    Function to add class shapes to the ClassDiagram section of the XML.
    
    Args:
        shapes: The parent XML element 'Shapes' where class shapes will be added.
        pimClasses: DataFrame containing class details such as ClassIdRef, Name, and MetaModelElement.
    """
    
    pimClasses=generateXYValues(pimClasses, pimRelations)
    # Iterate over each row in the pimClasses DataFrame
    for index, row in pimClasses.iterrows():
        classIdRef = row['ClassIdRef']
        className = row['Class Name']
        xValue = row['X']
        yValue = row['Y']
        zOrderValue = row['ZOrder']
        
        # Add the Class shape for each class
        classShape = ET.SubElement(shapes, 'Class', {
            "AttributeSortType": "0",
            "Background": "rgb(245, 245, 245)",
            "ConnectToPoint": "true",
            "ConnectionPointType": "2",
            "CoverConnector": "false",
            "CreatorDiagramType": "ClassDiagram",
            "DisplayAsRobustnessAnalysisIcon": "true",
            "EnumerationLiteralSortType": "0",
            "Foreground": "rgb(102, 102, 102)",
            "Height": "40",  # Height is fixed for all classes
            "Id": classIdRef,  # The ID from pimClasses
            "InterfaceBall": "false",
            "KShDrOp": "false",
            "KSwCsMbSt": "true",
            "LShCmMl": "false",
            "LshRfAts": "0",
            "MShDrAt": "false",
            "MSwTpPts": "true",
            "MetaModelElement": classIdRef,  # Same as ClassIdRef
            "Model": classIdRef,  # Same as ClassIdRef
            "ModelElementNameAlignment": "1",
            "Name": className,  # The class name from pimClasses
            "OperationSortType": "0",
            "OverrideAppearanceWithStereotypeIcon": "true",
            "ParentConnectorDTheta": "0.0",
            "ParentConnectorHeaderLength": "40",
            "ParentConnectorLineLength": "10",
            "PresentationOption": "4",
            "PrimitiveShapeType": "0",
            "ReceptionSortType": "0",
            "RequestDefaultSize": "false",
            "RequestFitSize": "false",
            "RequestFitSizeFromCenter": "false",
            "RequestResetCaption": "false",
            "RequestResetCaptionFitWidth": "false",
            "RequestResetCaptionSize": "false",
            "RequestSetSizeOption": "0",
            "Selectable": "true",
            "ShowAllocatedFrom": "0",
            "ShowAllocatedTo": "0",
            "ShowAttributeType": "1",
            "ShowAttributesCodeDetails": "0",
            "ShowAttributesPropertyModifiers": "0",
            "ShowAttributesType": "0",
            "ShowClassMemberConstraints": "true",
            "ShowEllipsisForUnshownMembers": "0",
            "ShowEmptyCompartments": "0",
            "ShowEnumerationLiteralType": "1",
            "ShowInitialAttributeValue": "true",
            "ShowOperationParameterDirection": "false",
            "ShowOperationProperties": "false",
            "ShowOperationRaisedExceptions": "false",
            "ShowOperationSignature": "true",
            "ShowOperationTemplateParameters": "false",
            "ShowOperationType": "1",
            "ShowOperationsCodeDetails": "0",
            "ShowOperationsParameters": "0",
            "ShowOperationsReturnType": "0",
            "ShowOwnerOption": "3",
            "ShowParameterNameInOperationSignature": "true",
            "ShowParametersCodeDetails": "0",
            "ShowReceptionType": "1",
            "ShowStereotypeIconName": "0",
            "ShowTypeOption": "0",
            "SuppressImplied1MultiplicityForAttribute": "0",
            "VisibilityStyle": "1",
            "Width": "163",  # Fixed width for all classes
            "WpMbs": "false",
            "X": str(xValue),  # X value from pimClasses
            "Y": str(yValue),  # Y value from pimClasses
            "ZOrder": str(zOrderValue)  # Z value from pimClasses
        })
        
        # Add additional sub-elements such as ElementFont, Line, Caption, FillColor, etc.
        ET.SubElement(classShape, "ElementFont", {
            "Color": "rgb(0, 0, 0)",
            "Name": "Dialog",
            "Size": "14",
            "Style": "0"
        })
        
        line = ET.SubElement(classShape, "Line", {
            "Cap": "0",
            "Color": "rgb(102, 102, 102)",
            "Transparency": "0",
            "Weight": "1.0"
        })
        ET.SubElement(line, "Stroke")
        
        ET.SubElement(classShape, "Caption", {
            "Height": "19",
            "InternalHeight": "-2147483648",
            "InternalWidth": "-2147483648",
            "Side": "FreeMove",
            "Visible": "true",
            "Width": "164",
            "X": "0",
            "Y": "0"
        })
        
        ET.SubElement(classShape, "FillColor", {
            "Color": "rgb(245, 245, 245)",
            "Style": "1",
            "Transparency": "0",
            "Type": "1"
        })
        
        ET.SubElement(classShape, "CompartmentFont", {
            "Value": "none"
        })

    return shapes


In [263]:
def addSubElementsToConnector(connector_element, relation_type, from_x, from_y, to_x, to_y):
    """
    Function to add sub-elements like ElementFont, Line, Caption, Points, and 
    specific elements for Association, Generalization, Usage, and Dependency connectors.
    
    Args:
        connector_element: The XML element for the connector.
        relation_type: The type of relationship (Generalization, Association, Usage, Dependency).
        from_x, from_y: Coordinates of the "From" class.
        to_x, to_y: Coordinates of the "To" class.
    """
    # Add common sub-elements: ElementFont, Line, and Caption
    ET.SubElement(connector_element, "ElementFont", {
        "Color": "rgb(0, 0, 0)",
        "Name": "Dialog",
        "Size": "16" if relation_type != 'Association' else "12",  # Different font size for association
        "Style": "0"
    })

    line = ET.SubElement(connector_element, "Line", {
        "Cap": "0",
        "Color": "rgb(0, 0, 0)",
        "Transparency": "0",
        "Weight": "1.0"
    })
    ET.SubElement(line, "Stroke")

    caption_x = (from_x + to_x) / 2
    caption_y = (from_y + to_y) / 2
    ET.SubElement(connector_element, "Caption", {
        "Height": "0" if relation_type != 'Dependency' else "21",  # For Dependency, Caption height is different
        "InternalHeight": "-2147483648",
        "InternalWidth": "-2147483648",
        "Side": "None",
        "Visible": "true",
        "Width": "20" if relation_type != 'Dependency' else "160",  # For Dependency, Caption width is different
        "X": str(caption_x),
        "Y": str(caption_y + 20) if relation_type == 'Dependency' else str(caption_y)
    })

    # Add Points to describe the path of the connector between classes
    points = ET.SubElement(connector_element, "Points")
    ET.SubElement(points, "Point", {"X": str(from_x + 20), "Y": str(from_y + 50)})  # Starting point near "From" class
    ET.SubElement(points, "Point", {"X": str(to_x), "Y": str(to_y - 20)})          # Ending point near "To" class

    # Additional sub-elements based on the relation type
    if relation_type == 'Association':
        # Add specific sub-elements for Association
        ET.SubElement(connector_element, "MultiplicityBRectangle", {
            "Height": "16",
            "Width": "39",
            "X": str(to_x + 50),
            "Y": str(to_y + 30)
        })
        
        roleB = ET.SubElement(connector_element, "RoleB")
        ET.SubElement(roleB, "MultiplicityCaption", {
            "Height": "16",
            "Width": "39",
            "X": str(to_x + 50),
            "Y": str(to_y + 30)
        })

    elif relation_type == 'Usage':
        # Add specific sub-elements for Usage (mostly similar to Generalization)
        # No special elements for Usage beyond the common ones
        pass

    elif relation_type == 'Dependency':
        # Add specific sub-elements for Dependency
        pass  # Dependency only requires the standard elements


def addConnectorsForRelations(connectors, pimRelations, pimClasses):
    """
    Function to add connectors for relations in the ClassDiagram section, including Dependency relations.
    
    Args:
        connectors: Class diagram XML element to which the connectors will be added.
        pimRelations: DataFrame containing relationships between classes.
        pimClasses: DataFrame containing class details like ClassIdRef.
    """
    
    zOrderCounter = 100  # Start ZOrder for connectors

    # Iterate over each relation in pimRelations DataFrame
    for _, relation in pimRelations.iterrows():
        from_class_id = relation['From Class ID']
        to_class_id = relation['To Class ID']
        relation_type = relation['Relationship Type']
        shape_id = relation['ShapeID']

        # Get X, Y positions of From and To classes from pimClasses
        from_class_position = pimClasses[pimClasses['Class ID'] == from_class_id][['X', 'Y']].values[0]
        to_class_position = pimClasses[pimClasses['Class ID'] == to_class_id][['X', 'Y']].values[0]
        
        from_x, from_y = from_class_position
        to_x, to_y = to_class_position

        # Generate the midpoint for connector X, Y values
        x_value = (from_x + to_x) / 2
        y_value = (from_y + to_y) / 2
        z_order_value = zOrderCounter
        zOrderCounter += 1

        from_class_id = pimClasses[pimClasses['Class ID'] == from_class_id]['ClassIdRef'].values[0]
        to_class_id = pimClasses[pimClasses['Class ID'] == to_class_id]['ClassIdRef'].values[0]
        # Common attributes for all connectors
        common_attributes = {
            "Background": "rgb(255, 255, 255)" if relation_type == 'Generalization' else "rgb(122, 207, 245)",  # Different colors for generalization vs others
            "ConnectorLabelOrientation": "4",
            "ConnectorLineJumps": "4",
            "ConnectorStyle": "Follow Diagram",
            "CreatorDiagramType": "ClassDiagram",
            "Foreground": "rgb(0, 0, 0)",
            "From": from_class_id,
            "FromConnectType": "0",
            "FromPinType": "1",
            "FromShapeXDiff": "0",
            "FromShapeYDiff": "0",
            "Height": str(abs(from_y - to_y)),
            "Id": shape_id,
            "MetaModelElement": shape_id,
            "Model": shape_id,
            "ModelElementNameAlignment": "9",
            "PaintThroughLabel": "2",
            "RequestRebuild": "false",
            "Selectable": "true",
            "ShowConnectorName": "2",
            "To": to_class_id,
            "ToConnectType": "0",
            "ToPinType": "1",
            "ToShapeXDiff": "0",
            "ToShapeYDiff": "0",
            "UseFromShapeCenter": "true",
            "UseToShapeCenter": "true",
            "Width": str(abs(from_x - to_x)),
            "X": str(x_value),
            "Y": str(y_value),
            "ZOrder": str(z_order_value)
        }


        # Create the connector element based on the relation type
        if relation_type == 'Generalization':
            connector_element = ET.SubElement(connectors, 'Generalization', common_attributes)
        elif relation_type == 'Aggregation' or relation_type == 'Association':
            connector_element = ET.SubElement(connectors, 'Association', common_attributes)
        elif relation_type == 'Usage':
            connector_element = ET.SubElement(connectors, 'Usage', common_attributes)
        elif relation_type == 'Dependency':  # Add Dependency relation
            common_attributes["Name"] = "&lt;&lt;compliant with&gt;&gt;"  # Name field for Dependency relation
            connector_element = ET.SubElement(connectors, 'Dependency', common_attributes)

        # Add the sub-elements like ElementFont, Line, Caption, Points, and specific elements for each relation type
        addSubElementsToConnector(connector_element, relation_type, from_x, from_y, to_x, to_y)

    return connectors


In [ ]:
XMLPath = './outputpim.xml'
projectAuthor="aless_xtbdjh4"
projectName = "PIMDT"
project = ET.Element("Project", Author=projectAuthor, 
                     CommentTableSortAscending="false", 
                     CommentTableSortColumn="Date Time", 
                     DocumentationType="html", 
                     ExportedFromDifferentName="false", 
                     ExporterVersion="16.1.1", 
                     Name=projectName, 
                     TextualAnalysisHighlightOptionCaseSensitive="false", 
                     UmlVersion="2.x", 
                     Xml_structure="simple")
 
projectInfo = ET.SubElement(project,"ProjectInfo")

logicalView = ET.SubElement(projectInfo, "LogicalView")
addStaticProjectOptions(projectInfo)


models = ET.SubElement(project, 'Models')
addStaticDataType(models, projectAuthor)

### MODEL RELATIONS CONTAINER ####
existingIDs= getExistingIds(pimClasses)
IdLength=getIdLength(pimClasses)

usageStereotypeIdRef = addUsageStereotype(projectAuthor, models, existingIDs, IdLength)
IDContainer = generateId(existingIDs, IdLength)
relationContainer = ET.SubElement(models, 'ModelRelationshipContainer', 
                                  BacklogActivityId="0", 
                                  Documentation_plain="", 
                                  Id=IDContainer, 
                                  Name="relationships", 
                                  PmAuthor=projectAuthor, 
                                  PmCreateDateTime="2024-09-05T12:10:58.354", 
                                  PmLastModified="2024-09-09T10:55:31.670", 
                                  QualityReason_IsNull="true", 
                                  QualityScore="-1", 
                                  UserIDLastNumericValue="0", 
                                  UserID_IsNull="true")

modelChildren = ET.SubElement(relationContainer, 'ModelChildren')
modelChildren = addUsageRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, usageStereotypeIdRef, existingIDs, IdLength)
modelChildren = addAssociationRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, IdLength)
modelChildren = addGeneralizationRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, IdLength)
modelChildren = addDependencyRelationContainer(projectAuthor, modelChildren, pimRelations, pimClasses, existingIDs, IdLength)
#################################


### CLASS  ####
models = addClassElement(models, projectAuthor, pimClasses, pimRelations, existingIDs, IdLength)
###############

############## DIAGRAM #############
diagrams = ET.SubElement(project, 'Diagrams')
classDiagramID = generateId(existingIDs, IdLength)
classDiagramName = 'BolognaDigitalTwin'
classDiagram = addClassDiagram(diagrams, projectAuthor, classDiagramID, classDiagramName)
shapes = ET.SubElement(classDiagram, 'Shapes')
shapes = addClassShapes(shapes, pimClasses, pimRelations)
connectors = ET.SubElement(classDiagram, 'Connectors')
connectors = addConnectorsForRelations(connectors, pimRelations, pimClasses)

formattedXML = prettify(project)
writeXML(project, XMLPath)
print(formattedXML)


In [ ]:
print("\nUpdated PIM Classes after starting writing XML:")
print(pimClasses.to_string(index=False))

print("\nUpdated PIM Relationships after starting writing XML:")
print(pimRelations.to_string(index=False))

